In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
import scipy
from scipy import stats
import os


In [ ]:
def corr_scatter(data, x, y, title): 
    
    xlim = data.loc[:,x].min() 
    #print(xlim)
    ylim = data.loc[:,y].max() - data.loc[:,y].std()
    #print(ylim)
    sns.lmplot(x=x, y=y, data=data, scatter_kws={"s": 6})
    per = stats.pearsonr(data[x], data[y])
    plt.title(title) 
    plt.text(xlim, ylim, 'Pearson corr {} \np_value {:e}'.format(round(per[0],3),per[1]), horizontalalignment='left', size='large', color='black')
    if x == 'PEBP1':
        plt.xlabel('RKIP')
    if y == 'PEBP1':
        plt.ylabel('RKIP')

    plt.show()


In [ ]:
def flotify(data):
    c = data.columns
    for i in c:
        data[i] = data[i].astype('float')


def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

def ind_change(data):
    ind = list(data.index)
    indn = []
    for i in ind:
        indn.append(i.strip())
    indn
    data.index = indn
    return data




def KS_Score_Finder(Dataset):
    a = pd.DataFrame([[0]*len(Dataset.T)])
    a.columns = Dataset.columns
    a.index = ['KS_Score']
    a.loc['KS_Mes_Score'] = 0 
    a.loc['KS_Mes_p_Value'] = 0 
    a.loc['KS_Epi_Score'] = 0 
    a.loc['KS_Epi_p_value'] = 0 

    for i in Dataset.columns:
        KS_MesGrt = scipy.stats.ks_2samp(Dataset.loc[intersection(Dataset.index, KS_Tumor_Mes),].loc[:,i],Dataset.loc[intersection(Dataset.index, KS_Tumor_Epi),].loc[:,i], alternative='greater')
        KS_EpiGrt = scipy.stats.ks_2samp(Dataset.loc[intersection(Dataset.index, KS_Tumor_Epi),].loc[:,i],Dataset.loc[intersection(Dataset.index, KS_Tumor_Mes),].loc[:,i], alternative='greater')
        a.loc['KS_Mes_Score',i] = KS_MesGrt[0]
        a.loc['KS_Mes_p_Value',i] = KS_MesGrt[1]
        a.loc['KS_Epi_Score',i] = KS_EpiGrt[0]
        a.loc['KS_Epi_p_value',i] = KS_EpiGrt[1]
        #print(i)


    for i in Dataset.columns:
        #print(i)
        if a.loc['KS_Mes_p_Value',i] < 0.05:
            a.loc['KS_Score',i] = -1*a.loc['KS_Mes_Score',i]
        
        elif a.loc['KS_Epi_p_value',i] < 0.05:
            a.loc['KS_Score',i] = a.loc['KS_Epi_Score',i]
        
        elif a.loc['KS_Epi_Score',i] > a.loc['KS_Mes_Score',i]:
            a.loc['KS_Score',i] = a.loc['KS_Epi_Score',i]
        
        else: 
            a.loc['KS_Score',i] = -1*a.loc['KS_Mes_Score',i]
        
    return a

In [ ]:
KS_Tumor_Epi = list(pd.read_csv('Tumor_KS_Epi.gmt', sep='\t'))[2:]
KS_Tumor_Mes = list(pd.read_csv('Tumor_KS_Mes.gmt', sep='\t'))[2:]


In [ ]:
def GS76_Score_TPM_input(data):
    li = sorted(list(pd.read_csv('76GS_Genes_original.txt', sep = '\t').T.loc['Gene',:]))
    #print(li)
    genes_in_dataset = data.index
    filt = intersection(li, genes_in_dataset)
    #print(len(filt))
    
    weights = []
    filtered_list = data.loc[filt,].sort_index()
    weight = pd.DataFrame([[0]*len(filt)])
    weight.columns = filtered_list.index
    #print(weight)
    

    for i in filtered_list.index:
        if i !='CDH1':
            weight.loc[0,i]= stats.pearsonr(filtered_list.loc['CDH1',], filtered_list.loc[i,])[0]
            #print(weights)
        elif i == 'CDH1':
            weight.loc[0,i]=0
    #print(len(weights))

    #weighted_array = np.array(weights) 
    GS76_score = ind_change(filtered_list).T@weight.T
    GS76_Centered = (GS76_score - GS76_score.mean()).T


    for i in data.columns:
        data.loc['76GS_Score',i] = GS76_Centered.loc[0,i] 

    return data


In [ ]:
import gseapy
def SigScore(dataset):

    dat = gseapy.ssgsea(data= dataset, gene_sets= "GenSig_RKIP_Metastasis.gmt", outdir = 'test', min_size = 5)
    dataset  = dataset.reindex(sorted(dataset.columns), axis=1)
    GenSig_RKIP_Metastasis_scores = pd.read_csv('test/gseapy.gene_set.ssgsea.report.csv', index_col=0)
    GenSig_RKIP_Metastasis_scores = GenSig_RKIP_Metastasis_scores.sort_values(by = 'Name', ascending = True)
    dataset.loc['GenSig_RKIP_Metastasis_scores',:] = GenSig_RKIP_Metastasis_scores['NES']

    dat = gseapy.ssgsea(data= dataset, gene_sets= "Tumor_KS_Epi.gmt", outdir = 'test', min_size = 5)
    dataset  = dataset.reindex(sorted(dataset.columns), axis=1)
    GenSig_Tumor_KS_Epi = pd.read_csv('test/gseapy.gene_set.ssgsea.report.csv', index_col=0)
    GenSig_Tumor_KS_Epi = GenSig_Tumor_KS_Epi.sort_values(by = 'Name', ascending = True)
    dataset.loc['KS_Epithelial_Gene_Signature',:] = GenSig_Tumor_KS_Epi['NES']

    dat = gseapy.ssgsea(data= dataset, gene_sets= "Tumor_KS_Mes.gmt", outdir = 'test', min_size = 5)
    dataset  = dataset.reindex(sorted(dataset.columns), axis=1)
    GenSig_Tumor_KS_Mes = pd.read_csv('test/gseapy.gene_set.ssgsea.report.csv', index_col=0)
    GenSig_Tumor_KS_Mes = GenSig_Tumor_KS_Mes.sort_values(by = 'Name', ascending = True)
    dataset.loc['KS_Mesenchymal_Gene_Signature',:] = GenSig_Tumor_KS_Mes['NES']

    dat = gseapy.ssgsea(data= dataset, gene_sets= "Hallmark_OXPHOS.gmt", outdir = 'test', min_size = 5)
    dataset  = dataset.reindex(sorted(dataset.columns), axis=1)
    GenSig_Hallmark_OXPHOS = pd.read_csv('test/gseapy.gene_set.ssgsea.report.csv', index_col=0)
    GenSig_Hallmark_OXPHOS = GenSig_Hallmark_OXPHOS.sort_values(by = 'Name', ascending = True)
    dataset.loc['Hallmark_OXPHOS',:] = GenSig_Hallmark_OXPHOS['NES']

    dat = gseapy.ssgsea(data= dataset, gene_sets= "Hallmark_Glycolysis.gmt", outdir = 'test', min_size = 5)
    dataset  = dataset.reindex(sorted(dataset.columns), axis=1)
    GenSig_Hallmark_Glycolysis = pd.read_csv('test/gseapy.gene_set.ssgsea.report.csv', index_col=0)
    GenSig_Hallmark_Glycolysis = GenSig_Hallmark_Glycolysis.sort_values(by = 'Name', ascending = True)
    dataset.loc['Hallmark_Glycolysis',:] = GenSig_Hallmark_Glycolysis['NES']

    dat = gseapy.ssgsea(data= dataset, gene_sets= "GenSig_PD-L1.gmt", outdir = 'test', min_size = 5)
    dataset  = dataset.reindex(sorted(dataset.columns), axis=1)
    GenSig_PDL1 = pd.read_csv('test/gseapy.gene_set.ssgsea.report.csv', index_col=0)
    GenSig_PDL1 = GenSig_PDL1.sort_values(by = 'Name', ascending = True)
    dataset.loc['PD-L1_Gene_Signature',:] = GenSig_PDL1['NES']

    dat = gseapy.ssgsea(data= dataset, gene_sets= "GenSig_FAO.gmt", outdir = 'test', min_size = 5)
    dataset  = dataset.reindex(sorted(dataset.columns), axis=1)
    GenSig_FAO = pd.read_csv('test/gseapy.gene_set.ssgsea.report.csv', index_col=0)
    GenSig_FAO = GenSig_FAO.sort_values(by = 'Name', ascending = True)
    dataset.loc['FAO_Gene_Signature',:] = GenSig_FAO['NES']

    dat = gseapy.ssgsea(data= dataset, gene_sets= "BACH1_Metastasis.gmt", outdir = 'test', min_size = 5)
    dataset  = dataset.reindex(sorted(dataset.columns), axis=1)
    GenSig_BACH1_Metastasis_scores = pd.read_csv('test/gseapy.gene_set.ssgsea.report.csv', index_col=0)
    GenSig_BACH1_Metastasis_scores = GenSig_BACH1_Metastasis_scores.sort_values(by = 'Name', ascending = True)
    dataset.loc['GenSig_BACH1_Metastasis_scores',:] = GenSig_BACH1_Metastasis_scores['NES']

    dat = gseapy.ssgsea(data= dataset, gene_sets= "GenSig_Ferroptosis_Immune_45.gmt", outdir = 'test', min_size = 5)
    dataset  = dataset.reindex(sorted(dataset.columns), axis=1)
    GenSig_Ferroptosis_Immune_45 = pd.read_csv('test/gseapy.gene_set.ssgsea.report.csv', index_col=0)
    GenSig_Ferroptosis_Immune_45 = GenSig_Ferroptosis_Immune_45.sort_values(by = 'Name', ascending = True)
    dataset.loc['GenSig_Ferroptosis_Immune_45',:] = GenSig_Ferroptosis_Immune_45['NES']

    return dataset


In [ ]:
'''import gseapy
def SigScore2(dataset):

    dat = gseapy.ssgsea(data= dataset, gene_sets= "NRF2_Meta_GenSig_Colorectal.gmt", outdir = 'test', min_size = 5)
    dataset  = dataset.reindex(sorted(dataset.columns), axis=1)
    GenSig_NRF2_scores = pd.read_csv('test/gseapy.gene_set.ssgsea.report.csv', index_col=0)
    GenSig_NRF2_scores = GenSig_NRF2_scores.sort_values(by = 'Name', ascending = True)
    dataset.loc['NRF2_Gene_Signature',:] = GenSig_NRF2_scores['NES']

    dat = gseapy.ssgsea(data= dataset, gene_sets= "GenSig_Ferroptosis_6_Mphag_bladder.gmt", outdir = 'test', min_size = 5)
    dataset  = dataset.reindex(sorted(dataset.columns), axis=1)
    GenSig_Ferroptosis_6 = pd.read_csv('test/gseapy.gene_set.ssgsea.report.csv', index_col=0)
    GenSig_Ferroptosis_6 = GenSig_Ferroptosis_6.sort_values(by = 'Name', ascending = True)
    dataset.loc['GenSig_Ferroptosis_6',:] = GenSig_Ferroptosis_6['NES']

    dat = gseapy.ssgsea(data= dataset, gene_sets= "GenSig_Ferroptosis_10_LBC.gmt", outdir = 'test', min_size = 5)
    dataset  = dataset.reindex(sorted(dataset.columns), axis=1)
    GenSig_Ferroptosis_10_LBC = pd.read_csv('test/gseapy.gene_set.ssgsea.report.csv', index_col=0)
    GenSig_Ferroptosis_10_LBC = GenSig_Ferroptosis_10_LBC.sort_values(by = 'Name', ascending = True)
    dataset.loc['GenSig_Ferroptosis_10_LBC',:] = GenSig_Ferroptosis_10_LBC['NES']

    dat = gseapy.ssgsea(data= dataset, gene_sets= "GenSig_Ferroptosis_Immune_45.gmt", outdir = 'test', min_size = 5)
    dataset  = dataset.reindex(sorted(dataset.columns), axis=1)
    GenSig_Ferroptosis_Immune_45 = pd.read_csv('test/gseapy.gene_set.ssgsea.report.csv', index_col=0)
    GenSig_Ferroptosis_Immune_45 = GenSig_Ferroptosis_Immune_45.sort_values(by = 'Name', ascending = True)
    dataset.loc['GenSig_Ferroptosis_Immune_45',:] = GenSig_Ferroptosis_Immune_45['NES']

    return dataset
'''

In [ ]:
TCGA_BRCA_scored= pd.read_csv('Data_Figure1_2/TCGA_BRCA.csv', index_col=0)

In [ ]:
TCGA_ACC_scored = pd.read_csv('Data_Figure1_2/TCGA_ACC.csv', index_col=0)
TCGA_BLCA_scored = pd.read_csv('Data_Figure1_2/TCGA_BLCA.csv', index_col=0)
TCGA_BRCA_scored= pd.read_csv('Data_Figure1_2/TCGA_BRCA.csv', index_col=0)
TCGA_CESC_scored =pd.read_csv('Data_Figure1_2/TCGA_CESC.csv', index_col=0)
TCGA_CHOL_scored= pd.read_csv('Data_Figure1_2/TCGA_CHOL.csv', index_col=0)
TCGA_COAD_scored= pd.read_csv('Data_Figure1_2/TCGA_COAD.csv', index_col=0)
TCGA_COADREAD_scored = pd.read_csv('Data_Figure1_2/TCGA_COADREAD.csv', index_col=0)
TCGA_ESCA_scored= pd.read_csv('Data_Figure1_2/TCGA_ESCA.csv', index_col=0)
TCGA_GBM_scored= pd.read_csv('Data_Figure1_2/TCGA_GBM.csv', index_col=0)
TCGA_GBMLGG_scored= pd.read_csv('Data_Figure1_2/TCGA_GBMLGG.csv', index_col=0)
TCGA_HNSC_scored= pd.read_csv('Data_Figure1_2/TCGA_HNSC.csv', index_col=0)
TCGA_KICH_scored= pd.read_csv('Data_Figure1_2/TCGA_KICH.csv', index_col=0)
TCGA_KIRC_scored= pd.read_csv('Data_Figure1_2/TCGA_KIRC.csv', index_col=0)
TCGA_KIRP_scored= pd.read_csv('Data_Figure1_2/TCGA_KIRP.csv', index_col=0)
TCGA_LAML_scored= pd.read_csv('Data_Figure1_2/TCGA_LAML.csv', index_col=0)
TCGA_LGG_scored= pd.read_csv('Data_Figure1_2/TCGA_LGG.csv', index_col=0)
TCGA_LIHC_scored= pd.read_csv('Data_Figure1_2/TCGA_LIHC.csv', index_col=0)
TCGA_LUAD_scored= pd.read_csv('Data_Figure1_2/TCGA_LUAD.csv', index_col=0)
TCGA_LUNG_scored= pd.read_csv('Data_Figure1_2/TCGA_LUNG.csv', index_col=0)
TCGA_LUSC_scored= pd.read_csv('Data_Figure1_2/TCGA_LUSC.csv', index_col=0)
TCGA_MESO_scored= pd.read_csv('Data_Figure1_2/TCGA_MESO.csv', index_col=0)
TCGA_OV_scored= pd.read_csv('Data_Figure1_2/TCGA_OV.csv', index_col=0)
TCGA_PAAD_scored= pd.read_csv('Data_Figure1_2/TCGA_PAAD.csv', index_col=0)
TCGA_PCPG_scored= pd.read_csv('Data_Figure1_2/TCGA_PCPG.csv', index_col=0)
TCGA_PRAD_scored= pd.read_csv('Data_Figure1_2/TCGA_PRAD.csv', index_col=0)
TCGA_READ_scored= pd.read_csv('Data_Figure1_2/TCGA_READ.csv', index_col=0)
TCGA_SARC_scored= pd.read_csv('Data_Figure1_2/TCGA_SARC.csv', index_col=0)
TCGA_SKCM_scored= pd.read_csv('Data_Figure1_2/TCGA_SKCM.csv', index_col=0)
TCGA_STAD_scored= pd.read_csv('Data_Figure1_2/TCGA_STAD.csv', index_col=0)
TCGA_TGCT_scored= pd.read_csv('Data_Figure1_2/TCGA_TCGT.csv', index_col=0)
TCGA_THCA_scored= pd.read_csv('Data_Figure1_2/TCGA_THCA.csv', index_col=0)
TCGA_THYM_scored= pd.read_csv('Data_Figure1_2/TCGA_THYM.csv', index_col=0)
TCGA_UCEC_scored= pd.read_csv('Data_Figure1_2/TCGA_UCEC.csv', index_col=0)
TCGA_UCS_scored= pd.read_csv('Data_Figure1_2/TCGA_USC.csv', index_col=0)
TCGA_UVM_scored= pd.read_csv('Data_Figure1_2/TCGA_UVM.csv', index_col=0)

Figure 2

Figure S2

In [ ]:
def corr_scatter(data, x, y, title, xlim='A',ylim='A', dpi=100 , xlabel='A', ylabel='A'): 
    per = stats.spearmanr(data[x], data[y])
    if xlim == "A":
        if per[0]>0:
            xlim = data.loc[:,x].min() 
        else:
            xlim = data.loc[:,x].max() - data.loc[:,x].mean() 
        
    if ylim == "A":
        if per[0]>0:
            ylim = data.loc[:,y].max() 
        else:
            ylim = data.loc[:,y].min() 

    print(xlim)
#    if ylim == "A":
#        ylim = data.loc[:,y].max() - data.loc[:,y].std()
    print(ylim)
    plt.rcParams['figure.dpi']= dpi
    sns.lmplot(x=x, y=y, data=data, scatter_kws={"s": 6 })
    per = stats.spearmanr(data[x], data[y])
    plt.title(title)
    
    if per[1] <0.0001:
        
        plt.text(xlim, ylim, 'Spearman corr {} \n p_value < 1e-04'.format(round(per[0],3)), horizontalalignment='left', size=13, color='Black', fontname='Arial')
    else:
        plt.text(xlim, ylim, 'Spearman corr {} \n p_value {}'.format(round(per[0],2),round(per[1],3)), horizontalalignment='left', size=13, color='Black',  fontname='Arial')
    
    plt.ylabel(y,fontsize=14, fontweight='bold', fontname='Arial')
    plt.xlabel(x,fontsize=14, fontweight='bold',fontname='Arial')
    
    if x == 'PEBP1':
        plt.xlabel('RKIP',fontsize=14)
    if y == 'PEBP1':
        plt.ylabel('RKIP',fontsize=14)

    if x == 'CD274':
        plt.xlabel('PDL1',fontsize=14)
    if y == 'CD274':
        plt.ylabel('PDL1',fontsize=14)

    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

    if xlabel != 'A':
        plt.xlabel(xlabel, fontsize=14) 
    if ylabel != 'A':
        plt.ylabel(ylabel, fontsize=14) 

    plt.show()


S2A (ii) and (v)

In [ ]:
corr_scatter(TCGA_BRCA_scored.T, 'PEBP1', 'VIM','',10,11, dpi=100)
corr_scatter(TCGA_BRCA_scored.T, 'BACH1', 'VIM','', 6,17, dpi=100)

S2A (iii) and (vi)

In [ ]:
corr_scatter(TCGA_BRCA_scored.T, 'PEBP1', 'OVOL2', '', 10,2,dpi=100)
corr_scatter(TCGA_BRCA_scored.T, 'BACH1', 'OVOL2','',6,2, dpi=100)

S2A (i) and (iv)

In [ ]:
corr_scatter(TCGA_BRCA_scored.T, 'BACH1', 'SNAI2', '',6,12,100)
corr_scatter(TCGA_BRCA_scored.T, 'PEBP1', 'SNAI2', '',10,4,100)


In [ ]:
'''corr_scatter(TCGA_BRCA_scored.T, 'PEBP1', 'CDH1', '',1, dpi=100)
corr_scatter(TCGA_BRCA_scored.T, 'BACH1', 'CDH1', '',1,dpi=100)'''
'''corr_scatter(TCGA_BRCA_scored.T, 'PEBP1', 'KLF4', '', 10,5,dpi=100)
corr_scatter(TCGA_BRCA_scored.T, 'BACH1', 'KLF4','',6,13, dpi=100)'''
'''corr_scatter(TCGA_BRCA_scored.T, 'PEBP1', 'ELF3', '', 10,2,dpi=100)
corr_scatter(TCGA_BRCA_scored.T, 'BACH1', 'ELF3','',6,2, dpi=100)'''
'''corr_scatter(TCGA_BRCA_scored.T, 'PEBP1', 'TWIST1', '', 10,2,dpi=100)
corr_scatter(TCGA_BRCA_scored.T, 'BACH1', 'TWIST1','',6,12, dpi=100)'''
'''corr_scatter(TCGA_BRCA_scored.T, 'PEBP1', 'GRHL2', '', 10,2,dpi=100)
corr_scatter(TCGA_BRCA_scored.T, 'BACH1', 'GRHL2','',6,2, dpi=100)'''
'''corr_scatter(TCGA_BRCA_scored.T, 'PEBP1', 'ESR2', '',10.2,7,100)
corr_scatter(TCGA_BRCA_scored.T, 'BACH1', 'ESR2', '',6,7,100)'''

In [ ]:
'''TCGA_BRCA_scored.T[TCGA_BRCA_scored.T.loc[: , 'ESR1'] > 8.9]'''

In [ ]:
def EM_kde_plot(Network, EM_upper , EM_lower ):
    plt.figure(figsize=(20, 30))
    #sns.set_palette(palette)
    sns.displot(Network, x="ESR1", fill=True, bins = 120 ,kde = True, color = 'red')
    plt.rcParams['figure.dpi']= 100

    plt.axvline(x = EM_lower, color = 'black', linestyle = ':')
    plt.axvline(x = EM_upper, color = 'black', linestyle = ':')

    plt.xticks(fontsize = 14, fontname='Arial')
    plt.yticks(fontsize = 14, fontname='Arial')
    plt.xlabel('ESR1',fontsize = 14, fontname='Arial',  fontweight='bold')

    plt.ylabel('Frequency',fontsize = 14, fontname='Arial',  fontweight='bold')

In [ ]:
'''EM_kde_plot(TCGA_BRCA_scored.T, 8.9,0 )

corr_scatter(TCGA_BRCA_scored.T[TCGA_BRCA_scored.T.loc[: , 'ESR1'] > 8.9], 'PEBP1', 'ESR1', '',10.3,9,200)
corr_scatter(TCGA_BRCA_scored.T[TCGA_BRCA_scored.T.loc[: , 'ESR1'] > 8.9], 'BACH1', 'ESR1', '',5.7,9,200)
corr_scatter(TCGA_BRCA_scored.T, 'PEBP1', 'BACH1', '',10, 6, 200)'''

Figure 2B

In [ ]:
corr_scatter(TCGA_BRCA_scored.T, 'PEBP1', 'ESR1', '',9.8,15, 1000)
corr_scatter(TCGA_BRCA_scored.T, 'BACH1', 'ESR1', '',5.8,3, 1000)

Figure 2B

In [ ]:
corr_scatter(TCGA_BRCA_scored.T, 'BACH1', 'ZEB1', '',5.8,12, 150)
corr_scatter(TCGA_BRCA_scored.T, 'PEBP1', 'ZEB1', '',10,5, 150)

Figure 2C

In [ ]:
corr_scatter(TCGA_BRCA_scored.T, 'BACH1', 'GenSig_BACH1_Metastasis_scores', '', dpi=150, xlabel='BACH1', ylabel='BACH1 Pathway Metastasis Signature ')

corr_scatter(TCGA_BRCA_scored.T, 'PEBP1', 'GenSig_RKIP_Metastasis_scores', '', 9.8, 0.6,dpi=150, xlabel='RKIP', ylabel='RKIP Pathway Metastasis Signature')

Figure 2D

In [ ]:

corr_scatter(TCGA_BRCA_scored.T, 'PD-L1_Gene_Signature', 'GenSig_BACH1_Metastasis_scores', '', 0.2,-0.8 ,dpi=1000, xlabel = 'PD-L1 Gene Signature' , ylabel = 'BACH1 Pathway Metastasis Signature' )
corr_scatter(TCGA_BRCA_scored.T, 'PD-L1_Gene_Signature', 'GenSig_RKIP_Metastasis_scores', '',0.2,0.6, dpi=1000, xlabel = 'PD-L1 Gene Signature' ,  ylabel = 'RKIP Pathway Metastasis Signature' )


In [ ]:
TCGA_BRCA_scored

In [ ]:
'''corr_scatter(TCGA_BRCA_scored.T, 'PEBP1', 'CD274', '',9.8,1)
corr_scatter(TCGA_BRCA_scored.T, 'BACH1', 'CD274', '',6,8)
corr_scatter(TCGA_BRCA_scored.T, 'GenSig_BACH1_Metastasis_scores','BACH1', '')
corr_scatter(TCGA_BRCA_scored.T, 'GenSig_BACH1_Metastasis_scores','PEBP1', '', -0.8, 10 )
corr_scatter(TCGA_BRCA_scored.T, 'GenSig_RKIP_Metastasis_scores','PEBP1', '', 0.6, 10 )
corr_scatter(TCGA_BRCA_scored.T, 'GenSig_RKIP_Metastasis_scores','BACH1', '', 1.2, 6 )
corr_scatter(TCGA_BRCA_scored.T, 'PD-L1_Gene_Signature','GenSig_BACH1_Metastasis_scores', '', -0.3, 0.15 )
corr_scatter(TCGA_BRCA_scored.T, 'PD-L1_Gene_Signature','GenSig_RKIP_Metastasis_scores', '', -0.3, 1.55 )
corr_scatter(TCGA_BRCA_scored.T, 'PD-L1_Gene_Signature','GenSig_RKIP_Metastasis_scores', '', -0.3, 1.55 )'''

In [ ]:
TCGA_ACC_scored = SigScore(TCGA_ACC_scored)
TCGA_BLCA_scored = SigScore(TCGA_BLCA_scored)
TCGA_BRCA_scored = SigScore(TCGA_BRCA_scored)
TCGA_CESC_scored = SigScore(TCGA_CESC_scored)
TCGA_CHOL_scored = SigScore(TCGA_CHOL_scored)
TCGA_COAD_scored = SigScore(TCGA_COAD_scored)
TCGA_COADREAD_scored = SigScore(TCGA_COADREAD_scored)
TCGA_ESCA_scored = SigScore(TCGA_ESCA_scored)
TCGA_GBM_scored = SigScore(TCGA_GBM_scored)
TCGA_GBMLGG_scored = SigScore(TCGA_GBMLGG_scored)
TCGA_HNSC_scored = SigScore(TCGA_HNSC_scored)
TCGA_KICH_scored = SigScore(TCGA_KICH_scored)
TCGA_KIRC_scored = SigScore(TCGA_KIRC_scored)
TCGA_KIRP_scored = SigScore(TCGA_KIRP_scored)
TCGA_LAML_scored = SigScore(TCGA_LAML_scored)
TCGA_LGG_scored = SigScore(TCGA_LGG_scored)
TCGA_LIHC_scored = SigScore(TCGA_LIHC_scored)
TCGA_LUAD_scored = SigScore(TCGA_LUAD_scored)
TCGA_LUNG_scored = SigScore(TCGA_LUNG_scored)
TCGA_LUSC_scored = SigScore(TCGA_LUSC_scored)
TCGA_MESO_scored = SigScore(TCGA_MESO_scored)
TCGA_OV_scored = SigScore(TCGA_OV_scored)
TCGA_PAAD_scored = SigScore(TCGA_PAAD_scored)
TCGA_PCPG_scored = SigScore(TCGA_PCPG_scored)
TCGA_PRAD_scored = SigScore(TCGA_PRAD_scored)
TCGA_READ_scored = SigScore(TCGA_READ_scored)
TCGA_SARC_scored = SigScore(TCGA_SARC_scored)
TCGA_SKCM_scored = SigScore(TCGA_SKCM_scored)
TCGA_STAD_scored = SigScore(TCGA_STAD_scored)
TCGA_TGCT_scored = SigScore(TCGA_TGCT_scored)
TCGA_THCA_scored = SigScore(TCGA_THCA_scored)
TCGA_THYM_scored = SigScore(TCGA_THYM_scored)
TCGA_UCEC_scored = SigScore(TCGA_UCEC_scored)
TCGA_UCS_scored = SigScore(TCGA_UCS_scored)
TCGA_UVM_scored = SigScore(TCGA_UVM_scored)

In [ ]:
'''TCGA_ACC_scored = SigScore2(TCGA_ACC_scored)
TCGA_BLCA_scored = SigScore2(TCGA_BLCA_scored)
TCGA_BRCA_scored = SigScore2(TCGA_BRCA_scored)
TCGA_CESC_scored = SigScore2(TCGA_CESC_scored)
TCGA_CHOL_scored = SigScore2(TCGA_CHOL_scored)
TCGA_COAD_scored = SigScore2(TCGA_COAD_scored)
TCGA_COADREAD_scored = SigScore2(TCGA_COADREAD_scored)
TCGA_ESCA_scored = SigScore2(TCGA_ESCA_scored)
TCGA_GBM_scored = SigScore2(TCGA_GBM_scored)
TCGA_GBMLGG_scored = SigScore2(TCGA_GBMLGG_scored)
TCGA_HNSC_scored = SigScore2(TCGA_HNSC_scored)
TCGA_KICH_scored = SigScore2(TCGA_KICH_scored)
TCGA_KIRC_scored = SigScore2(TCGA_KIRC_scored)
TCGA_KIRP_scored = SigScore2(TCGA_KIRP_scored)
TCGA_LAML_scored = SigScore2(TCGA_LAML_scored)
TCGA_LGG_scored = SigScore2(TCGA_LGG_scored)
TCGA_LIHC_scored = SigScore2(TCGA_LIHC_scored)
TCGA_LUAD_scored = SigScore2(TCGA_LUAD_scored)
TCGA_LUNG_scored = SigScore2(TCGA_LUNG_scored)
TCGA_LUSC_scored = SigScore2(TCGA_LUSC_scored)
TCGA_MESO_scored = SigScore2(TCGA_MESO_scored)
TCGA_OV_scored = SigScore2(TCGA_OV_scored)
TCGA_PAAD_scored = SigScore2(TCGA_PAAD_scored)
TCGA_PCPG_scored = SigScore2(TCGA_PCPG_scored)
TCGA_PRAD_scored = SigScore2(TCGA_PRAD_scored)
TCGA_READ_scored = SigScore2(TCGA_READ_scored)
TCGA_SARC_scored = SigScore2(TCGA_SARC_scored)
TCGA_SKCM_scored = SigScore2(TCGA_SKCM_scored)
TCGA_STAD_scored = SigScore2(TCGA_STAD_scored)
TCGA_TGCT_scored = SigScore2(TCGA_TGCT_scored)
TCGA_THCA_scored = SigScore2(TCGA_THCA_scored)
TCGA_THYM_scored = SigScore2(TCGA_THYM_scored)
TCGA_UCEC_scored = SigScore2(TCGA_UCEC_scored)
TCGA_UCS_scored = SigScore2(TCGA_UCS_scored)
TCGA_UVM_scored = SigScore2(TCGA_UVM_scored)'''

In [ ]:
'''

(SigScore(TCGA_ACC_scored)).to_csv('Data_Figure1_2/TCGA_ACC.csv')
(SigScore(TCGA_BLCA_scored)).to_csv('Data_Figure1_2/TCGA_BLCA.csv')
(SigScore(TCGA_BRCA_scored)).to_csv('Data_Figure1_2/TCGA_BRCA.csv')
(SigScore(TCGA_CESC_scored)).to_csv('Data_Figure1_2/TCGA_CESC.csv')
(SigScore(TCGA_CHOL_scored)).to_csv('Data_Figure1_2/TCGA_CHOL.csv')
(SigScore(TCGA_COAD_scored)).to_csv('Data_Figure1_2/TCGA_COAD.csv')
(SigScore(TCGA_COADREAD_scored)).to_csv('Data_Figure1_2/TCGA_COADREAD.csv')
(SigScore(TCGA_ESCA_scored)).to_csv('Data_Figure1_2/TCGA_ESCA.csv')
(SigScore(TCGA_GBM_scored)).to_csv('Data_Figure1_2/TCGA_GBM.csv')
(SigScore(TCGA_GBMLGG_scored)).to_csv('Data_Figure1_2/TCGA_GBMLGG.csv')
(SigScore(TCGA_HNSC_scored)).to_csv('Data_Figure1_2/TCGA_HNSC.csv')
(SigScore(TCGA_KICH_scored)).to_csv('Data_Figure1_2/TCGA_KICH.csv')
(SigScore(TCGA_KIRC_scored)).to_csv('Data_Figure1_2/TCGA_KIRC.csv')
(SigScore(TCGA_KIRP_scored)).to_csv('Data_Figure1_2/TCGA_KIRP.csv')
(SigScore(TCGA_LAML_scored)).to_csv('Data_Figure1_2/TCGA_LAML.csv')
(SigScore(TCGA_LGG_scored)).to_csv('Data_Figure1_2/TCGA_LGG.csv')
(SigScore(TCGA_LIHC_scored)).to_csv('Data_Figure1_2/TCGA_LIHC.csv')
(SigScore(TCGA_LUAD_scored)).to_csv('Data_Figure1_2/TCGA_LUAD.csv')
(SigScore(TCGA_LUNG_scored)).to_csv('Data_Figure1_2/TCGA_LUNG.csv')
(SigScore(TCGA_LUSC_scored)).to_csv('Data_Figure1_2/TCGA_LUSC.csv')
(SigScore(TCGA_MESO_scored)).to_csv('Data_Figure1_2/TCGA_MESO.csv')
(SigScore(TCGA_OV_scored)).to_csv('Data_Figure1_2/TCGA_OV.csv')
(SigScore(TCGA_PAAD_scored)).to_csv('Data_Figure1_2/TCGA_PAAD.csv')
(SigScore(TCGA_PCPG_scored)).to_csv('Data_Figure1_2/TCGA_PCPG.csv')
(SigScore(TCGA_PRAD_scored)).to_csv('Data_Figure1_2/TCGA_PRAD.csv')
(SigScore(TCGA_READ_scored)).to_csv('Data_Figure1_2/TCGA_READ.csv')
(SigScore(TCGA_SARC_scored)).to_csv('Data_Figure1_2/TCGA_SARC.csv')
(SigScore(TCGA_SKCM_scored)).to_csv('Data_Figure1_2/TCGA_SKCM.csv')
(SigScore(TCGA_STAD_scored)).to_csv('Data_Figure1_2/TCGA_STAD.csv')
(SigScore(TCGA_TGCT_scored)).to_csv('Data_Figure1_2/TCGA_TCGT.csv')
(SigScore(TCGA_THCA_scored)).to_csv('Data_Figure1_2/TCGA_THCA.csv')
(SigScore(TCGA_THYM_scored)).to_csv('Data_Figure1_2/TCGA_THYM.csv')
(SigScore(TCGA_UCEC_scored)).to_csv('Data_Figure1_2/TCGA_UCEC.csv')
(SigScore2(TCGA_UCS_scored)).to_csv('Data_Figure1_2/TCGA_USC.csv')
(SigScore2(TCGA_UVM_scored)).to_csv('Data_Figure1_2/TCGA_UVM.csv')
'''


In [ ]:
'''TCGA_BLCA_scored = SigScore2(TCGA_BLCA_scored)
TCGA_BLCA_scored.tail(20)
#corr_scatter(TCGA_BLCA_scored.T, 'GenSig_Ferroptosis_Immune_45','GenSig_RKIP_Metastasis_scores',)
corr_scatter(TCGA_BLCA_scored.T, 'BACH1','NFE2L2','')'''

In [ ]:
(SigScore(TCGA_ACC_scored)).to_csv('Data_Figure1_2/TCGA_ACC.csv')
(SigScore(TCGA_BLCA_scored)).to_csv('Data_Figure1_2/TCGA_BLCA.csv')
(SigScore(TCGA_BRCA_scored)).to_csv('Data_Figure1_2/TCGA_BRCA.csv')
(SigScore(TCGA_CESC_scored)).to_csv('Data_Figure1_2/TCGA_CESC.csv')
(SigScore(TCGA_CHOL_scored)).to_csv('Data_Figure1_2/TCGA_CHOL.csv')
(SigScore(TCGA_COAD_scored)).to_csv('Data_Figure1_2/TCGA_COAD.csv')
(SigScore(TCGA_COADREAD_scored)).to_csv('Data_Figure1_2/TCGA_COADREAD.csv')
(SigScore(TCGA_ESCA_scored)).to_csv('Data_Figure1_2/TCGA_ESCA.csv')
(SigScore(TCGA_GBM_scored)).to_csv('Data_Figure1_2/TCGA_GBM.csv')
(SigScore(TCGA_GBMLGG_scored)).to_csv('Data_Figure1_2/TCGA_GBMLGG.csv')
(SigScore(TCGA_HNSC_scored)).to_csv('Data_Figure1_2/TCGA_HNSC.csv')
(SigScore(TCGA_KICH_scored)).to_csv('Data_Figure1_2/TCGA_KICH.csv')
(SigScore(TCGA_KIRC_scored)).to_csv('Data_Figure1_2/TCGA_KIRC.csv')
(SigScore(TCGA_KIRP_scored)).to_csv('Data_Figure1_2/TCGA_KIRP.csv')
(SigScore(TCGA_LAML_scored)).to_csv('Data_Figure1_2/TCGA_LAML.csv')
(SigScore(TCGA_LGG_scored)).to_csv('Data_Figure1_2/TCGA_LGG.csv')
(SigScore(TCGA_LIHC_scored)).to_csv('Data_Figure1_2/TCGA_LIHC.csv')
(SigScore(TCGA_LUAD_scored)).to_csv('Data_Figure1_2/TCGA_LUAD.csv')
(SigScore(TCGA_LUNG_scored)).to_csv('Data_Figure1_2/TCGA_LUNG.csv')
(SigScore(TCGA_LUSC_scored)).to_csv('Data_Figure1_2/TCGA_LUSC.csv')
(SigScore(TCGA_MESO_scored)).to_csv('Data_Figure1_2/TCGA_MESO.csv')
(SigScore(TCGA_OV_scored)).to_csv('Data_Figure1_2/TCGA_OV.csv')
(SigScore(TCGA_PAAD_scored)).to_csv('Data_Figure1_2/TCGA_PAAD.csv')
(SigScore(TCGA_PCPG_scored)).to_csv('Data_Figure1_2/TCGA_PCPG.csv')
(SigScore(TCGA_PRAD_scored)).to_csv('Data_Figure1_2/TCGA_PRAD.csv')
(SigScore(TCGA_READ_scored)).to_csv('Data_Figure1_2/TCGA_READ.csv')
(SigScore(TCGA_SARC_scored)).to_csv('Data_Figure1_2/TCGA_SARC.csv')
(SigScore(TCGA_SKCM_scored)).to_csv('Data_Figure1_2/TCGA_SKCM.csv')
(SigScore(TCGA_STAD_scored)).to_csv('Data_Figure1_2/TCGA_STAD.csv')
(SigScore(TCGA_TGCT_scored)).to_csv('Data_Figure1_2/TCGA_TCGT.csv')
(SigScore(TCGA_THCA_scored)).to_csv('Data_Figure1_2/TCGA_THCA.csv')
(SigScore(TCGA_THYM_scored)).to_csv('Data_Figure1_2/TCGA_THYM.csv')
(SigScore(TCGA_UCEC_scored)).to_csv('Data_Figure1_2/TCGA_UCEC.csv')
(SigScore2(TCGA_UCS_scored)).to_csv('Data_Figure1_2/TCGA_USC.csv')
(SigScore2(TCGA_UVM_scored)).to_csv('Data_Figure1_2/TCGA_UVM.csv')

In [ ]:
'''Corr_Tab_Gly = pd.DataFrame([[0]*17]*35).T

Corr_Tab_Gly.index = ['RKIP vs BACH1','RKIP vs PDL1 Signature' , 'BACH1 vs PDL1 Signature', 'RKIP vs Hallmark Glycolysis', 'BACH1 vs Hallmark Glycolysis', 'RKIP vs FAO Gene Signature', 'BACH1 vs FAO Gene Signature', 'RKIP vs Hallmark OXPHOS', 'BACH1 vs Hallmark OXPHOS', 'RKIP vs KS_Score (Tumor)', 'BACH1 vs KS_Score (Tumor)', 'RKIP vs KS_Epithelial_Gene_Signature', 'BACH1 vs KS_Epithelial_Gene_Signature','RKIP vs KS_Mesenchymal_Gene_Signature', 'BACH1 vs KS_Mesenchymal_Gene_Signature','RKIP vs ESR1','BACH1 vs ESR1']
Corr_Tab_Gly.columns = ['TCGA_ACC', 'TCGA_BLCA', 'TCGA_BRCA', 'TCGA_CESC','TCGA_CHOL', 'TCGA_COAD','TCGA_COADREAD','TCGA_ESCA','TCGA_GBM','TCGA_GBMLGG','TCGA_HNSC','TCGA_KICH','TCGA_KIRC','TCGA_KIRP','TCGA_LAML','TCGA_LGG','TCGA_LIHC','TCGA_LUAD','TCGA_LUNG','TCGA_LUSC','TCGA_MESO','TCGA_OV','TCGA_PAAD','TCGA_PCPG','TCGA_PRAD','TCGA_READ','TCGA_SARC','TCGA_SKCM','TCGA_STAD','TCGA_TGCT','TCGA_THCA','TCGA_THYM','TCGA_UCEC','TCGA_UCS','TCGA_UVM']
Corr_Tab_Gly'''

In [ ]:
'''tcga_datasets = [TCGA_ACC_scored,TCGA_BLCA_scored, TCGA_BRCA_scored, TCGA_CESC_scored, 
TCGA_CHOL_scored,TCGA_COAD_scored,TCGA_COADREAD_scored,TCGA_ESCA_scored,TCGA_GBM_scored ,TCGA_GBMLGG_scored, TCGA_HNSC_scored ,
TCGA_KICH_scored ,TCGA_KIRC_scored,TCGA_KIRP_scored ,TCGA_LAML_scored,
TCGA_LGG_scored,TCGA_LIHC_scored,TCGA_LUAD_scored ,TCGA_LUNG_scored ,
TCGA_LUSC_scored ,TCGA_MESO_scored ,TCGA_OV_scored,TCGA_PAAD_scored, 
TCGA_PCPG_scored ,TCGA_PRAD_scored ,TCGA_READ_scored ,TCGA_SARC_scored ,
TCGA_SKCM_scored ,TCGA_STAD_scored ,TCGA_TGCT_scored ,TCGA_THCA_scored,
TCGA_THYM_scored ,TCGA_UCEC_scored,TCGA_UCS_scored ,TCGA_UVM_scored ] 

tcga_labels = Corr_Tab_Gly.columns'''



In [ ]:
'''def corr_table(data, y):
    Corr_Tab_Gly.loc['RKIP vs BACH1', y]  = stats.pearsonr(data['PEBP1'], data['BACH1'])[0]
    Corr_Tab_Gly.loc['RKIP vs PDL1 Signature', y]  = stats.pearsonr(data['PEBP1'], data['PD-L1_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['BACH1 vs PDL1 Signature', y]  = stats.pearsonr(data['BACH1'], data['PD-L1_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['RKIP vs Hallmark Glycolysis', y]  = stats.pearsonr(data['PEBP1'], data['Hallmark_Glycolysis'])[0]
    Corr_Tab_Gly.loc['BACH1 vs Hallmark Glycolysis', y]  = stats.pearsonr( data['BACH1'], data['Hallmark_Glycolysis'])[0]
    Corr_Tab_Gly.loc['RKIP vs FAO Gene Signature', y]  = stats.pearsonr(data['PEBP1'], data['FAO_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['BACH1 vs FAO Gene Signature', y]  = stats.pearsonr(data['BACH1'], data['FAO_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['RKIP vs Hallmark OXPHOS', y]  = stats.pearsonr(data['PEBP1'], data['Hallmark_OXPHOS'])[0]
    Corr_Tab_Gly.loc['BACH1 vs Hallmark OXPHOS', y]  = stats.pearsonr(data['BACH1'], data['Hallmark_OXPHOS'])[0]
    Corr_Tab_Gly.loc['RKIP vs KS_Score (Tumor)', y]  = stats.pearsonr( data['PEBP1'], data['KS_Score (Tumor)'])[0]
    Corr_Tab_Gly.loc['BACH1 vs KS_Score (Tumor)', y]  = stats.pearsonr(data['BACH1'], data['KS_Score (Tumor)'])[0]
    Corr_Tab_Gly.loc['RKIP vs KS_Epithelial_Gene_Signature', y]  = stats.pearsonr(data['PEBP1'], data['KS_Epithelial_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['BACH1 vs KS_Epithelial_Gene_Signature', y]  = stats.pearsonr(data['BACH1'], data['KS_Epithelial_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['RKIP vs KS_Mesenchymal_Gene_Signature', y]  = stats.pearsonr(data['PEBP1'], data['KS_Mesenchymal_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['BACH1 vs KS_Mesenchymal_Gene_Signature', y]  = stats.pearsonr( data['BACH1'], data['KS_Mesenchymal_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['RKIP vs ESR1', y]  = stats.pearsonr(data['PEBP1'], data['ESR1'])[0]
    Corr_Tab_Gly.loc['BACH1 vs ESR1', y]  = stats.pearsonr( data['BACH1'], data['ESR1'])[0]'''

In [ ]:
'''
for i in range(len(tcga_datasets)) :
    corr_table(tcga_datasets[i].T, tcga_labels[i])'''

In [ ]:
'''plt.figure(figsize=(22,12))
#sns.set(rc={'figure.figsize':(10,10)})
plt.title('Pan Cancer Analysis (Spearman Co-efficients)')
sns.heatmap(round(Corr_Tab_Gly,2), linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.9, vmin =-0.9, annot = True
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.15, 'aspect': 40} )
#plt.ylabel('Gene/Pathway',  labelpad = 20)'''

In [ ]:
'''def corr_table_p_value(data, y, Corr_Tab_Gly ):
    Corr_Tab_Gly.loc['RKIP vs BACH1', y]  = stats.pearsonr(data['PEBP1'], data['BACH1'])[1]
    Corr_Tab_Gly.loc['RKIP vs PDL1 Signature', y]  = stats.pearsonr(data['PEBP1'], data['PD-L1_Gene_Signature'])[1]
    Corr_Tab_Gly.loc['BACH1 vs PDL1 Signature', y]  = stats.pearsonr(data['BACH1'], data['PD-L1_Gene_Signature'])[1]
    Corr_Tab_Gly.loc['RKIP vs Hallmark Glycolysis', y]  = stats.pearsonr(data['PEBP1'], data['Hallmark_Glycolysis'])[1]
    Corr_Tab_Gly.loc['BACH1 vs Hallmark Glycolysis', y]  = stats.pearsonr( data['BACH1'], data['Hallmark_Glycolysis'])[1]
    Corr_Tab_Gly.loc['RKIP vs FAO Gene Signature', y]  = stats.pearsonr(data['PEBP1'], data['FAO_Gene_Signature'])[1]
    Corr_Tab_Gly.loc['BACH1 vs FAO Gene Signature', y]  = stats.pearsonr(data['BACH1'], data['FAO_Gene_Signature'])[1]

    Corr_Tab_Gly.loc['RKIP vs Hallmark OXPHOS', y]  = stats.pearsonr(data['PEBP1'], data['Hallmark_OXPHOS'])[1]
    Corr_Tab_Gly.loc['BACH1 vs Hallmark OXPHOS', y]  = stats.pearsonr(data['BACH1'], data['Hallmark_OXPHOS'])[1]
    Corr_Tab_Gly.loc['RKIP vs KS_Score (Tumor)', y]  = stats.pearsonr( data['PEBP1'], data['KS_Score (Tumor)'])[1]
    Corr_Tab_Gly.loc['BACH1 vs KS_Score (Tumor)', y]  = stats.pearsonr(data['BACH1'], data['KS_Score (Tumor)'])[1]
    Corr_Tab_Gly.loc['RKIP vs KS_Epithelial_Gene_Signature', y]  = stats.pearsonr(data['PEBP1'], data['KS_Epithelial_Gene_Signature'])[1]
    Corr_Tab_Gly.loc['BACH1 vs KS_Epithelial_Gene_Signature', y]  = stats.pearsonr(data['BACH1'], data['KS_Epithelial_Gene_Signature'])[1]
    Corr_Tab_Gly.loc['RKIP vs KS_Mesenchymal_Gene_Signature', y]  = stats.pearsonr(data['PEBP1'], data['KS_Mesenchymal_Gene_Signature'])[1]
    Corr_Tab_Gly.loc['BACH1 vs KS_Mesenchymal_Gene_Signature', y]  = stats.pearsonr( data['BACH1'], data['KS_Mesenchymal_Gene_Signature'])[1]
    Corr_Tab_Gly.loc['RKIP vs ESR1', y]  = stats.pearsonr(data['PEBP1'], data['ESR1'])[1]
    Corr_Tab_Gly.loc['BACH1 vs ESR1', y]  = stats.pearsonr( data['BACH1'], data['ESR1'])[1]'''

'''Star_Corr_Tab_Gly = Corr_Tab_Gly.copy()
Star_Corr_Tab_Gly'''

'''
for i in range(len(tcga_datasets)) :
    corr_table_p_value(tcga_datasets[i].T, tcga_labels[i],Star_Corr_Tab_Gly )
    Star_Corr_Tab_Gly'''


In [ ]:
'''for i in range(len(Star_Corr_Tab_Gly.columns)):
    for j in range(len(Star_Corr_Tab_Gly.index)):
        
        if Star_Corr_Tab_Gly.iloc[j,i] < 0.05 :
            Star_Corr_Tab_Gly.iloc[j,i] = '*'
        else:
            Star_Corr_Tab_Gly.iloc[j,i] =
            
            Star_Corr_Tab_Gly'''
        
'''



In [ ]:
'''selected_rows_for_plot=['RKIP vs BACH1','RKIP vs KS_Mesenchymal_Gene_Signature','BACH1 vs KS_Mesenchymal_Gene_Signature',
                  'RKIP vs KS_Epithelial_Gene_Signature','BACH1 vs KS_Epithelial_Gene_Signature','RKIP vs KS_Score (Tumor)','BACH1 vs KS_Score (Tumor)']'''

In [ ]:
'''labelled=round(Corr_Tab_Gly.loc[selected_rows_for_plot,:],2)
labelled.index=['RKIP vs BACH1', 'RKIP vs KS_Mes_GenSig',
       'BACH1 vs KS_Mes_GenSig',
       'RKIP vs KS_Epi_GenSig',
       'BACH1 vs KS_Epi_GenSig', 'RKIP vs KS_Score (Tumor)',
       'BACH1 vs KS_Score (Tumor)']'''

In [ ]:
'''plt.figure(figsize=(30,20))
#sns.set(rc={'figure.figsize':(10,10)})
plt.title('Correlations of RKIP and BACH1 with metabolism gene signature across cancers (Spearman Co-efficients)')
sns.heatmap(labelled.T, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.8, vmin =-0.8, annot = Star_Corr_Tab_Gly.loc[selected_rows_for_plot,:].T
            ,cbar_kws={'orientation': 'vertical', 'pad' : 0.23, 'aspect': 40} )
#plt.ylabel('Gene/Pathway',  labelpad = 20)'''

Fig 1b

In [ ]:
'''score_table_pdl1 = round(Corr_Tab_Gly,2).loc[['RKIP vs PDL1 Signature','BACH1 vs PDL1 Signature'],:]
star_table_pdl1 = Star_Corr_Tab_Gly.loc[['RKIP vs PDL1 Signature','BACH1 vs PDL1 Signature'],:]'''

In [ ]:
'''score_table_pdl1.index =['RKIP vs PDL1 GenSig', 'BACH1 vs PDL1 GenSig']'''

In [ ]:
'''plt.figure(figsize=(15,8))
#sns.set(rc={'figure.figsize':(10,10)})
plt.title('Correlations of RKIP and BACH1 with PDL1 gene signature across cancers (Spearman Co-efficients)')

sns.heatmap(score_table_pdl1, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.8, vmin =-0.8, annot = star_table_pdl1,
            cbar_kws={'orientation': 'horizontal', 'pad' : 0.23, 'aspect': 40} )
plt.tick_params(labelsize=10)
#plt.ylabel('Gene/Pathway',  labelpad = 20)'''

fig 1c

In [ ]:
#selected_rows_for_plot_Meta_plot=['RKIP vs Hallmark OXPHOS','BACH1 vs Hallmark OXPHOS','RKIP vs FAO Gene Signature','BACH1 vs FAO Gene Signature','RKIP vs Hallmark Glycolysis','BACH1 vs Hallmark Glycolysis']

In [ ]:
#score_table_meta = round(Corr_Tab_Gly,2).loc[selected_rows_for_plot_Meta_plot,:]
#star_table_meta = Star_Corr_Tab_Gly.loc[selected_rows_for_plot_Meta_plot,:]

In [ ]:
##score_table_meta.index

In [ ]:
'''
score_table_meta.index=['RKIP vs OXPHOS_GenSig', 'BACH1 vs OXPHOS_GenSig',
       'RKIP vs FAO_GenSig', 'BACH1 vs FAO_GenSig',
       'RKIP vs Glycolysis_GenSig', 'BACH1 vs Glycolysis_GenSig']'''

In [ ]:
'''plt.figure(figsize=(15,8))
#sns.set(rc={'figure.figsize':(10,10)})
plt.title('Pan Cancer Analysis (Spearman Co-efficients)')
sns.heatmap(score_table_meta, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 1, vmin =-1, annot = star_table_meta
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.23, 'aspect': 40} )
#plt.ylabel('Gene/Pathway',  labelpad = 20)'''

In [ ]:
'''def corr_table(data, y):
    Corr_Tab_Gly.loc['RKIP vs BACH1', y]  = stats.pearsonr(data['PEBP1'], data['BACH1'])[0]
    Corr_Tab_Gly.loc['RKIP vs PDL1 Signature', y]  = stats.pearsonr(data['PEBP1'], data['PD-L1_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['BACH1 vs PDL1 Signature', y]  = stats.pearsonr(data['BACH1'], data['PD-L1_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['RKIP vs Hallmark Glycolysis', y]  = stats.pearsonr(data['PEBP1'], data['Hallmark_Glycolysis'])[0]
    Corr_Tab_Gly.loc['BACH1 vs Hallmark Glycolysis', y]  = stats.pearsonr( data['BACH1'], data['Hallmark_Glycolysis'])[0]
    Corr_Tab_Gly.loc['RKIP vs FAO Gene Signature', y]  = stats.pearsonr(data['PEBP1'], data['FAO_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['BACH1 vs FAO Gene Signature', y]  = stats.pearsonr(data['BACH1'], data['FAO_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['RKIP vs Hallmark OXPHOS', y]  = stats.pearsonr(data['PEBP1'], data['Hallmark_OXPHOS'])[0]
    Corr_Tab_Gly.loc['BACH1 vs Hallmark OXPHOS', y]  = stats.pearsonr(data['BACH1'], data['Hallmark_OXPHOS'])[0]
    Corr_Tab_Gly.loc['RKIP vs KS_Score (Tumor)', y]  = stats.pearsonr( data['PEBP1'], data['KS_Score (Tumor)'])[0]
    Corr_Tab_Gly.loc['BACH1 vs KS_Score (Tumor)', y]  = stats.pearsonr(data['BACH1'], data['KS_Score (Tumor)'])[0]
    Corr_Tab_Gly.loc['RKIP vs KS_Epithelial_Gene_Signature', y]  = stats.pearsonr(data['PEBP1'], data['KS_Epithelial_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['BACH1 vs KS_Epithelial_Gene_Signature', y]  = stats.pearsonr(data['BACH1'], data['KS_Epithelial_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['RKIP vs KS_Mesenchymal_Gene_Signature', y]  = stats.pearsonr(data['PEBP1'], data['KS_Mesenchymal_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['BACH1 vs KS_Mesenchymal_Gene_Signature', y]  = stats.pearsonr( data['BACH1'], data['KS_Mesenchymal_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['RKIP vs ESR1', y]  = stats.pearsonr(data['PEBP1'], data['ESR1'])[0]
    Corr_Tab_Gly.loc['BACH1 vs ESR1', y]  = stats.pearsonr( data['BACH1'], data['ESR1'])[0]'''

In [ ]:
'''Corr_Tab_Gly = pd.DataFrame([[0]*23]*35).T

Corr_Tab_Gly.index = ['RKIP vs BACH1','RKIP vs PDL1 Signature' , 'BACH1 vs PDL1 Signature', 'RKIP vs Hallmark Glycolysis', 'BACH1 vs Hallmark Glycolysis', 'RKIP vs FAO Gene Signature', 'BACH1 vs FAO Gene Signature', 'RKIP vs Hallmark OXPHOS', 'BACH1 vs Hallmark OXPHOS', 'RKIP vs KS_Score (Tumor)', 'BACH1 vs KS_Score (Tumor)', 'RKIP vs KS_Epithelial_Gene_Signature', 'BACH1 vs KS_Epithelial_Gene_Signature','RKIP vs KS_Mesenchymal_Gene_Signature', 'BACH1 vs KS_Mesenchymal_Gene_Signature','RKIP vs ESR1','BACH1 vs ESR1','BACH1 vs 76GS_Score','RKIP vs 76GS_Score' ,'76GS_Score vs KS_Score (Tumor)','Hallmark_OXPHOS vs Hallmark_Glycolysis', 'FAO_Gene_Signature vs Hallmark_OXPHOS','FAO_Gene_Signature vs Hallmark_Glycolysis']
Corr_Tab_Gly.columns = ['TCGA_ACC', 'TCGA_BLCA', 'TCGA_BRCA', 'TCGA_CESC','TCGA_CHOL', 'TCGA_COAD','TCGA_COADREAD','TCGA_ESCA','TCGA_GBM','TCGA_GBMLGG','TCGA_HNSC','TCGA_KICH','TCGA_KIRC','TCGA_KIRP','TCGA_LAML','TCGA_LGG','TCGA_LIHC','TCGA_LUAD','TCGA_LUNG','TCGA_LUSC','TCGA_MESO','TCGA_OV','TCGA_PAAD','TCGA_PCPG','TCGA_PRAD','TCGA_READ','TCGA_SARC','TCGA_SKCM','TCGA_STAD','TCGA_TGCT','TCGA_THCA','TCGA_THYM','TCGA_UCEC','TCGA_UCS','TCGA_UVM']
Corr_Tab_Gly'''

In [ ]:
'''def corr_table(data, y):
    Corr_Tab_Gly.loc['RKIP vs BACH1', y]  = stats.spearmanr(data['PEBP1'], data['BACH1'])[0]
    Corr_Tab_Gly.loc['RKIP vs PDL1 Signature', y]  = stats.spearmanr(data['PEBP1'], data['PD-L1_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['BACH1 vs PDL1 Signature', y]  = stats.spearmanr(data['BACH1'], data['PD-L1_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['RKIP vs Hallmark Glycolysis', y]  = stats.spearmanr(data['PEBP1'], data['Hallmark_Glycolysis'])[0]
    Corr_Tab_Gly.loc['BACH1 vs Hallmark Glycolysis', y]  = stats.spearmanr( data['BACH1'], data['Hallmark_Glycolysis'])[0]
    Corr_Tab_Gly.loc['RKIP vs FAO Gene Signature', y]  = stats.spearmanr(data['PEBP1'], data['FAO_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['BACH1 vs FAO Gene Signature', y]  = stats.spearmanr(data['BACH1'], data['FAO_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['RKIP vs Hallmark OXPHOS', y]  = stats.spearmanr(data['PEBP1'], data['Hallmark_OXPHOS'])[0]
    Corr_Tab_Gly.loc['BACH1 vs Hallmark OXPHOS', y]  = stats.spearmanr(data['BACH1'], data['Hallmark_OXPHOS'])[0]
    Corr_Tab_Gly.loc['RKIP vs KS_Score (Tumor)', y]  = stats.spearmanr( data['PEBP1'], data['KS_Score (Tumor)'])[0]
    Corr_Tab_Gly.loc['BACH1 vs KS_Score (Tumor)', y]  = stats.spearmanr(data['BACH1'], data['KS_Score (Tumor)'])[0]
    Corr_Tab_Gly.loc['RKIP vs KS_Epithelial_Gene_Signature', y]  = stats.spearmanr(data['PEBP1'], data['KS_Epithelial_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['BACH1 vs KS_Epithelial_Gene_Signature', y]  = stats.spearmanr(data['BACH1'], data['KS_Epithelial_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['RKIP vs KS_Mesenchymal_Gene_Signature', y]  = stats.spearmanr(data['PEBP1'], data['KS_Mesenchymal_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['BACH1 vs KS_Mesenchymal_Gene_Signature', y]  = stats.spearmanr( data['BACH1'], data['KS_Mesenchymal_Gene_Signature'])[0]
    Corr_Tab_Gly.loc['RKIP vs ESR1', y]  = stats.spearmanr(data['PEBP1'], data['ESR1'])[0]
    Corr_Tab_Gly.loc['BACH1 vs ESR1', y]  = stats.spearmanr(data['BACH1'], data['ESR1'])[0]
    Corr_Tab_Gly.loc['BACH1 vs 76GS_Score', y]  = stats.spearmanr( data['BACH1'], data['76GS_Score'])[0]
    Corr_Tab_Gly.loc['RKIP vs 76GS_Score', y]  = stats.spearmanr( data['PEBP1'], data['76GS_Score'])[0]
    Corr_Tab_Gly.loc['76GS_Score vs KS_Score (Tumor)', y]  = stats.spearmanr( data['76GS_Score'], data['KS_Score (Tumor)'])[0]
    Corr_Tab_Gly.loc['Hallmark_OXPHOS vs Hallmark_Glycolysis', y]  = stats.spearmanr( data['Hallmark_OXPHOS'], data['Hallmark_Glycolysis'])[0]
    Corr_Tab_Gly.loc['FAO_Gene_Signature vs Hallmark_OXPHOS', y]  = stats.spearmanr( data['FAO_Gene_Signature'], data['Hallmark_OXPHOS'])[0]
    Corr_Tab_Gly.loc['FAO_Gene_Signature vs Hallmark_Glycolysis', y]  = stats.spearmanr( data['FAO_Gene_Signature'], data['Hallmark_Glycolysis'])[0]'''

In [ ]:
'''
for i in range(len(tcga_datasets)) :
    corr_table(tcga_datasets[i].T, tcga_labels[i])
'''

In [ ]:
'''Corr_Tab_Gly_star=Corr_Tab_Gly.copy()
Corr_Tab_Gly_star'''

In [ ]:
Corr_Tab_Gly = pd.DataFrame([[0]*23]*35).T

Corr_Tab_Gly.index = ['RKIP vs BACH1','RKIP vs PDL1 Signature' , 'BACH1 vs PDL1 Signature', 'RKIP vs Hallmark Glycolysis', 'BACH1 vs Hallmark Glycolysis', 'RKIP vs FAO Gene Signature', 'BACH1 vs FAO Gene Signature', 'RKIP vs Hallmark OXPHOS', 'BACH1 vs Hallmark OXPHOS', 'RKIP vs KS_Score (Tumor)', 'BACH1 vs KS_Score (Tumor)', 'RKIP vs KS_Epithelial_Gene_Signature', 'BACH1 vs KS_Epithelial_Gene_Signature','RKIP vs KS_Mesenchymal_Gene_Signature', 'BACH1 vs KS_Mesenchymal_Gene_Signature','RKIP vs ESR1','BACH1 vs ESR1','BACH1 vs 76GS_Score','RKIP vs 76GS_Score' ,'76GS_Score vs KS_Score (Tumor)','Hallmark_OXPHOS vs Hallmark_Glycolysis', 'FAO_Gene_Signature vs Hallmark_OXPHOS','FAO_Gene_Signature vs Hallmark_Glycolysis']
Corr_Tab_Gly.columns = ['TCGA_ACC', 'TCGA_BLCA', 'TCGA_BRCA', 'TCGA_CESC','TCGA_CHOL', 'TCGA_COAD','TCGA_COADREAD','TCGA_ESCA','TCGA_GBM','TCGA_GBMLGG','TCGA_HNSC','TCGA_KICH','TCGA_KIRC','TCGA_KIRP','TCGA_LAML','TCGA_LGG','TCGA_LIHC','TCGA_LUAD','TCGA_LUNG','TCGA_LUSC','TCGA_MESO','TCGA_OV','TCGA_PAAD','TCGA_PCPG','TCGA_PRAD','TCGA_READ','TCGA_SARC','TCGA_SKCM','TCGA_STAD','TCGA_TGCT','TCGA_THCA','TCGA_THYM','TCGA_UCEC','TCGA_UCS','TCGA_UVM']
Corr_Tab_Gly


In [ ]:
tcga_datasets = [TCGA_ACC_scored,TCGA_BLCA_scored, TCGA_BRCA_scored, TCGA_CESC_scored, 
TCGA_CHOL_scored,TCGA_COAD_scored,TCGA_COADREAD_scored,TCGA_ESCA_scored,TCGA_GBM_scored ,TCGA_GBMLGG_scored, TCGA_HNSC_scored ,
TCGA_KICH_scored ,TCGA_KIRC_scored,TCGA_KIRP_scored ,TCGA_LAML_scored,
TCGA_LGG_scored,TCGA_LIHC_scored,TCGA_LUAD_scored ,TCGA_LUNG_scored ,
TCGA_LUSC_scored ,TCGA_MESO_scored ,TCGA_OV_scored,TCGA_PAAD_scored, 
TCGA_PCPG_scored ,TCGA_PRAD_scored ,TCGA_READ_scored ,TCGA_SARC_scored ,
TCGA_SKCM_scored ,TCGA_STAD_scored ,TCGA_TGCT_scored ,TCGA_THCA_scored,
TCGA_THYM_scored ,TCGA_UCEC_scored,TCGA_UCS_scored ,TCGA_UVM_scored ] 

tcga_labels = Corr_Tab_Gly.columns



In [ ]:
def corr_table(Corr_Tab_Gly, data, y,i):
    Corr_Tab_Gly.loc['RKIP vs BACH1', y]  = stats.spearmanr(data['PEBP1'], data['BACH1'])[i]
    Corr_Tab_Gly.loc['RKIP vs PDL1 Signature', y]  = stats.spearmanr(data['PEBP1'], data['PD-L1_Gene_Signature'])[i]
    Corr_Tab_Gly.loc['BACH1 vs PDL1 Signature', y]  = stats.spearmanr(data['BACH1'], data['PD-L1_Gene_Signature'])[i]
    Corr_Tab_Gly.loc['RKIP vs Hallmark Glycolysis', y]  = stats.spearmanr(data['PEBP1'], data['Hallmark_Glycolysis'])[i]
    Corr_Tab_Gly.loc['BACH1 vs Hallmark Glycolysis', y]  = stats.spearmanr( data['BACH1'], data['Hallmark_Glycolysis'])[i]
    Corr_Tab_Gly.loc['RKIP vs FAO Gene Signature', y]  = stats.spearmanr(data['PEBP1'], data['FAO_Gene_Signature'])[i]
    Corr_Tab_Gly.loc['BACH1 vs FAO Gene Signature', y]  = stats.spearmanr(data['BACH1'], data['FAO_Gene_Signature'])[i]
    Corr_Tab_Gly.loc['RKIP vs Hallmark OXPHOS', y]  = stats.spearmanr(data['PEBP1'], data['Hallmark_OXPHOS'])[i]
    Corr_Tab_Gly.loc['BACH1 vs Hallmark OXPHOS', y]  = stats.spearmanr(data['BACH1'], data['Hallmark_OXPHOS'])[i]
    Corr_Tab_Gly.loc['RKIP vs KS_Score (Tumor)', y]  = stats.spearmanr( data['PEBP1'], data['KS_Score (Tumor)'])[i]
    Corr_Tab_Gly.loc['BACH1 vs KS_Score (Tumor)', y]  = stats.spearmanr(data['BACH1'], data['KS_Score (Tumor)'])[i]
    Corr_Tab_Gly.loc['RKIP vs KS_Epithelial_Gene_Signature', y]  = stats.spearmanr(data['PEBP1'], data['KS_Epithelial_Gene_Signature'])[i]
    Corr_Tab_Gly.loc['BACH1 vs KS_Epithelial_Gene_Signature', y]  = stats.spearmanr(data['BACH1'], data['KS_Epithelial_Gene_Signature'])[i]
    Corr_Tab_Gly.loc['RKIP vs KS_Mesenchymal_Gene_Signature', y]  = stats.spearmanr(data['PEBP1'], data['KS_Mesenchymal_Gene_Signature'])[i]
    Corr_Tab_Gly.loc['BACH1 vs KS_Mesenchymal_Gene_Signature', y]  = stats.spearmanr( data['BACH1'], data['KS_Mesenchymal_Gene_Signature'])[i]
    Corr_Tab_Gly.loc['RKIP vs ESR1', y]  = stats.spearmanr(data['PEBP1'], data['ESR1'])[i]
    Corr_Tab_Gly.loc['BACH1 vs ESR1', y]  = stats.spearmanr(data['BACH1'], data['ESR1'])[i]
    Corr_Tab_Gly.loc['BACH1 vs 76GS_Score', y]  = stats.spearmanr( data['BACH1'], data['76GS_Score'])[i]
    Corr_Tab_Gly.loc['RKIP vs 76GS_Score', y]  = stats.spearmanr( data['PEBP1'], data['76GS_Score'])[i]
    Corr_Tab_Gly.loc['76GS_Score vs KS_Score (Tumor)', y]  = stats.spearmanr( data['76GS_Score'], data['KS_Score (Tumor)'])[i]
    Corr_Tab_Gly.loc['Hallmark_OXPHOS vs Hallmark_Glycolysis', y]  = stats.spearmanr( data['Hallmark_OXPHOS'], data['Hallmark_Glycolysis'])[i]
    Corr_Tab_Gly.loc['FAO_Gene_Signature vs Hallmark_OXPHOS', y]  = stats.spearmanr( data['FAO_Gene_Signature'], data['Hallmark_OXPHOS'])[i]
    Corr_Tab_Gly.loc['FAO_Gene_Signature vs Hallmark_Glycolysis', y]  = stats.spearmanr( data['FAO_Gene_Signature'], data['Hallmark_Glycolysis'])[i]
    Corr_Tab_Gly.loc['BACH1_Metastasis_GenSig vs RKIP', y]  = stats.spearmanr( data['GenSig_BACH1_Metastasis_scores'], data['PEBP1'])[i]
    Corr_Tab_Gly.loc['BACH1_Metastasis_GenSig vs BACH1', y]  = stats.spearmanr( data['GenSig_BACH1_Metastasis_scores'], data['BACH1'])[i]
    Corr_Tab_Gly.loc['RKIP_Metastasis_GenSig vs RKIP', y]  = stats.spearmanr( data['GenSig_RKIP_Metastasis_scores'], data['PEBP1'])[i]
    Corr_Tab_Gly.loc['RKIP_Metastasis_GenSig vs BACH1', y]  = stats.spearmanr( data['GenSig_RKIP_Metastasis_scores'], data['BACH1'])[i]
    Corr_Tab_Gly.loc['RKIP_Metastasis_GenSig vs BACH1_Metastasis_GenSig', y]  = stats.spearmanr( data['GenSig_RKIP_Metastasis_scores'], data['GenSig_BACH1_Metastasis_scores'])[i]
    Corr_Tab_Gly.loc['RKIP vs CD274(PDL1)', y]  = stats.spearmanr( data['PEBP1'], data['CD274'])[i]
    Corr_Tab_Gly.loc['BACH1 vs CD274(PDL1)', y]  = stats.spearmanr( data['BACH1'], data['CD274'])[i]
    
    #Corr_Tab_Gly.loc['RKIP vs NRF2', y]  = stats.spearmanr( data['PEBP1'], data['NFE2L2'])[i]
    #Corr_Tab_Gly.loc['BACH1 vs NRF2', y]  = stats.spearmanr( data['BACH1'], data['NFE2L2'])[i]
    
    #Corr_Tab_Gly.loc['RKIP vs NRF2_Gene_Signature', y]  = stats.spearmanr( data['PEBP1'], data['NRF2_Gene_Signature'])[i]
    #Corr_Tab_Gly.loc['BACH1 vs NRF2_Gene_Signature', y]  = stats.spearmanr( data['BACH1'], data['NRF2_Gene_Signature'])[i]
    #Corr_Tab_Gly.loc['NRF2 vs NRF2_Gene_Signature', y]  = stats.spearmanr( data['BACH1'], data['NFE2L2'])[i]

    #Corr_Tab_Gly.loc['RKIP vs GenSig_Ferroptosis_6', y]  = stats.spearmanr( data['PEBP1'], data['GenSig_Ferroptosis_6'])[i]
    #Corr_Tab_Gly.loc['BACH1 vs GenSig_Ferroptosis_6', y]  = stats.spearmanr( data['BACH1'], data['GenSig_Ferroptosis_6'])[i]
    #Corr_Tab_Gly.loc['NRF2 vs GenSig_Ferroptosis_6', y]  = stats.spearmanr( data['NFE2L2'], data['GenSig_Ferroptosis_6'])[i]

    #Corr_Tab_Gly.loc['RKIP vs GenSig_Ferroptosis_10_LBC', y]  = stats.spearmanr( data['PEBP1'], data['GenSig_Ferroptosis_10_LBC'])[i]
    #Corr_Tab_Gly.loc['BACH1 vs GenSig_Ferroptosis_10_LBC', y]  = stats.spearmanr( data['BACH1'], data['GenSig_Ferroptosis_10_LBC'])[i]
    #Corr_Tab_Gly.loc['NRF2 vs GenSig_Ferroptosis_10_LBC', y]  = stats.spearmanr( data['NFE2L2'], data['GenSig_Ferroptosis_10_LBC'])[i]


    Corr_Tab_Gly.loc['RKIP vs GenSig_Ferroptosis_Immune_45', y]  = stats.spearmanr( data['PEBP1'], data['GenSig_Ferroptosis_Immune_45'])[i]
    Corr_Tab_Gly.loc['BACH1 vs GenSig_Ferroptosis_Immune_45', y]  = stats.spearmanr( data['BACH1'], data['GenSig_Ferroptosis_Immune_45'])[i]
    Corr_Tab_Gly.loc['NEF2 vs GenSig_Ferroptosis_Immune_45', y]  = stats.spearmanr( data['NFE2L2'], data['GenSig_Ferroptosis_Immune_45'])[i]

    Corr_Tab_Gly.loc['RKIP vs VIM', y]  = stats.spearmanr( data['PEBP1'], data['VIM'])[i]
    Corr_Tab_Gly.loc['RKIP vs SNAI2', y]  = stats.spearmanr( data['PEBP1'], data['SNAI2'])[i]
    Corr_Tab_Gly.loc['RKIP vs OVOL2', y]  = stats.spearmanr( data['PEBP1'], data['OVOL2'])[i]

    
    Corr_Tab_Gly.loc['BACH1 vs VIM', y]  = stats.spearmanr( data['BACH1'], data['VIM'])[i]
    Corr_Tab_Gly.loc['BACH1 vs SNAI2', y]  = stats.spearmanr( data['BACH1'], data['SNAI2'])[i]
    Corr_Tab_Gly.loc['BACH1 vs OVOL2', y]  = stats.spearmanr( data['BACH1'], data['OVOL2'])[i]

In [ ]:

for i in range(len(tcga_datasets)):
    corr_table(Corr_Tab_Gly, tcga_datasets[i].T, tcga_labels[i], 1)


In [ ]:
Corr_Tab_Gly_star = Corr_Tab_Gly.copy()

In [ ]:

for i in range(len(tcga_datasets)):
    corr_table(Corr_Tab_Gly, tcga_datasets[i].T, tcga_labels[i], 0)


In [ ]:
Corr_Tab_Gly

Supplementary Figure 2 (Heatmap of scatterplot data)

In [ ]:
Corr_Tab_Gly.loc[['RKIP vs VIM','RKIP vs SNAI2','RKIP vs OVOL2','BACH1 vs VIM','BACH1 vs SNAI2','BACH1 vs OVOL2'],'TCGA_BRCA']
table_s2=pd.DataFrame(columns=['VIM','SNAI2','OVOL2'], index=['RKIP','BACH1'])
table_s2.loc['RKIP',:]= np.array(Corr_Tab_Gly.loc[['RKIP vs VIM','RKIP vs SNAI2','RKIP vs OVOL2'],'TCGA_BRCA'].T)
table_s2.loc['BACH1',:]= np.array(Corr_Tab_Gly.loc[['BACH1 vs VIM','BACH1 vs SNAI2','BACH1 vs OVOL2'],'TCGA_BRCA'].T)
table_s2=table_s2.astype(float)


In [ ]:
Corr_Tab_Gly_star.loc[['RKIP vs VIM','RKIP vs SNAI2','RKIP vs OVOL2','BACH1 vs VIM','BACH1 vs SNAI2','BACH1 vs OVOL2'],'TCGA_BRCA']
#Corr_Tab_Gly_star=Corr_Tab_Gly_star.T
table_s2_star=pd.DataFrame(columns=['VIM','SNAI2','OVOL2'], index=['RKIP','BACH1'])
table_s2_star=table_s2_star.T
table_s2_star.loc[:,'RKIP']= np.array(Corr_Tab_Gly_star.loc[['RKIP vs VIM','RKIP vs SNAI2','RKIP vs OVOL2'],'TCGA_BRCA'])
table_s2_star.loc[:,'BACH1']= np.array(Corr_Tab_Gly_star.loc[['BACH1 vs VIM','BACH1 vs SNAI2','BACH1 vs OVOL2'],'TCGA_BRCA'])
table_s2_star=table_s2_star.T
table_s2_star

In [ ]:
table_s2_stared=table_s2_star.copy()
table_s2_stared

In [ ]:
for i in range(len(table_s2_star.columns)):
    for j in range(len(table_s2_star.index)):
        
        if table_s2_stared.iloc[j,i] < 0.05 :
            table_s2_stared.iloc[j,i] = '*'
        else:
            table_s2_stared.iloc[j,i] = ''

table_s2_stared


In [ ]:
table_s2=table_s2.astype(float)
table_s2

In [ ]:
table_s2_stared.T

In [ ]:
plt.figure(figsize=(25,20))
plt.rcParams['figure.dpi']= 500
ax = sns.heatmap(table_s2, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.8, vmin = -0.8, annot = table_s2_stared
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.35 , 'aspect': 16, 'fraction' : 0.08},  
            annot_kws={'size': 100, 'verticalalignment':'center', 'horizontalalignment':'center'} )
plt.tick_params(axis='both', labelsize=80 )
cbar = ax.collections[0].colorbar
plt.xticks( rotation = 50, horizontalalignment='right')
plt.yticks( rotation = 0, horizontalalignment='right')
cbar.ax.tick_params(labelsize = 60)



In [ ]:
table_s2.T.loc[:,'RKIP']=Corr_Tab_Gly_star.loc[['RKIP vs VIM','RKIP vs SNAI2','RKIP vs OVOL2'],'TCGA_BRCA']

In [ ]:
np.array(Corr_Tab_Gly_star.loc[['RKIP vs VIM','RKIP vs SNAI2','RKIP vs OVOL2'],'TCGA_BRCA'])

In [ ]:
(Corr_Tab_Gly.T.loc[:,'RKIP vs KS_Epithelial_Gene_Signature'] > 0).value_counts()

In [ ]:
(Corr_Tab_Gly_star.T[Corr_Tab_Gly.T.loc[:,'RKIP vs KS_Epithelial_Gene_Signature'] > 0].loc[:,'RKIP vs KS_Epithelial_Gene_Signature']< 0.05).value_counts()

In [ ]:
(Corr_Tab_Gly.T.loc[:,'BACH1 vs KS_Mesenchymal_Gene_Signature'] > 0).value_counts()
(Corr_Tab_Gly_star.T[Corr_Tab_Gly.T.loc[:,'BACH1 vs KS_Mesenchymal_Gene_Signature'] > 0].loc[:,'BACH1 vs KS_Mesenchymal_Gene_Signature']< 0.05).value_counts()
(Corr_Tab_Gly_star.T[Corr_Tab_Gly.T.loc[:,'RKIP vs KS_Mesenchymal_Gene_Signature'] < 0].loc[:,'RKIP vs KS_Mesenchymal_Gene_Signature']< 0.05).value_counts()
(Corr_Tab_Gly.T.loc[:,'RKIP vs KS_Mesenchymal_Gene_Signature'] < 0).value_counts()
(Corr_Tab_Gly.T.loc[:,'RKIP vs KS_Score (Tumor)'] < 0).value_counts()
(Corr_Tab_Gly.T.loc[:,'BACH1 vs KS_Epithelial_Gene_Signature'] < 0).value_counts()
(Corr_Tab_Gly_star.T[Corr_Tab_Gly.T.loc[:,'BACH1 vs KS_Epithelial_Gene_Signature'] < 0].loc[:,'BACH1 vs KS_Epithelial_Gene_Signature']< 0.05).value_counts()
(Corr_Tab_Gly_star.T[Corr_Tab_Gly.T.loc[:,'RKIP vs KS_Score (Tumor)'] < 0].loc[:,'RKIP vs KS_Score (Tumor)']< 0.05).value_counts()
(Corr_Tab_Gly.T.loc[:,'BACH1 vs KS_Score (Tumor)'] > 0).value_counts()
(Corr_Tab_Gly_star.T[Corr_Tab_Gly.T.loc[:,'BACH1 vs KS_Score (Tumor)'] > 0].loc[:,'BACH1 vs KS_Score (Tumor)']< 0.05).value_counts()

In [ ]:
for i in range(len(Corr_Tab_Gly_star.columns)):
    for j in range(len(Corr_Tab_Gly_star.index)):
        
        if Corr_Tab_Gly_star.iloc[j,i] < 0.05 :
            Corr_Tab_Gly_star.iloc[j,i] = '*'
        else:
            Corr_Tab_Gly_star.iloc[j,i] = ''
        


In [ ]:
Corr_Tab_Gly

In [ ]:
plt.figure(figsize=(35,30))
#sns.set(rc={'figure.figsize':(10,10)})
plt.title('Pan Cancer Analysis (Spearman Co-efficients)')
sns.heatmap(Corr_Tab_Gly, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 1, vmin =-1, annot = Corr_Tab_Gly_star
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.23, 'aspect': 40} )
#plt.ylabel('Gene/Pathway',  labelpad = 20)

In [ ]:
plt.figure(figsize=(35,25))
#sns.set(rc={'figure.figsize':(10,10)})
plt.title('Pan Cancer Analysis (Spearman Co-efficients)')
sns.heatmap(Corr_Tab_Gly, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 1, vmin =-1, annot = Corr_Tab_Gly_star
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.23, 'aspect': 40} )
#plt.ylabel('Gene/Pathway',  labelpad = 20)

In [ ]:
Corr_Tab_Gly.columns = ['ACC', 'BLCA', 'BRCA', 'CESC', 'CHOL',
       'COAD', 'COADREAD', 'ESCA', 'GBM', 'GBMLGG',
       'HNSC', 'KICH', 'KIRC', 'KIRP', 'LAML',
       'LGG', 'LIHC', 'LUAD', 'LUNG', 'LUSC',
       'MESO', 'OV', 'PAAD', 'PCPG', 'PRAD',
       'READ', 'SARC', 'SKCM', 'STAD', 'TGCT',
       'THCA', 'THYM', 'UCEC', 'UCS', 'UVM']

In [ ]:
plt.figure(figsize=(35,25))
#sns.set(rc={'figure.figsize':(10,10)})
plt.title('Pan Cancer Analysis (Spearman Co-efficients)')
sns.heatmap(Corr_Tab_Gly, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 1, vmin =-1, annot = Corr_Tab_Gly_star
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.23, 'aspect': 40} )
#plt.ylabel('Gene/Pathway',  labelpad = 20)

Figure 1A


In [ ]:
'''fig_1_spear = Corr_Tab_Gly.loc[['RKIP vs BACH1','BACH1 vs KS_Mesenchymal_Gene_Signature','RKIP vs KS_Mesenchymal_Gene_Signature','BACH1 vs KS_Epithelial_Gene_Signature','RKIP vs KS_Epithelial_Gene_Signature','BACH1 vs KS_Score (Tumor)','RKIP vs KS_Score (Tumor)'],:]
fig_1_spear_star = Corr_Tab_Gly_star.loc[['RKIP vs BACH1','BACH1 vs KS_Mesenchymal_Gene_Signature','RKIP vs KS_Mesenchymal_Gene_Signature','BACH1 vs KS_Epithelial_Gene_Signature','RKIP vs KS_Epithelial_Gene_Signature','BACH1 vs KS_Score (Tumor)','RKIP vs KS_Score (Tumor)'],:]
fig_1_spear.index = ['RKIP vs BACH1','BACH1 vs KS_Mes (GS)','RKIP vs KS_Mes (GS)','BACH1 vs KS_Epi (GS)','RKIP vs KS_Epi (GS)','BACH1 vs KS_Score (Tumor)','RKIP vs KS_Score (Tumor)']

plt.figure(figsize=(30,20))
plt.rcParams['figure.dpi']= 1000
#sns.set(rc={'figure.figsize':(10,10)})

#plt.title('Correlations of RKIP and BACH1 with KS Gene Signatures (Spearman coefficients)', size=20)
ax = sns.heatmap(fig_1_spear.T, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.7, vmin =-0.7, annot = fig_1_spear_star.T
            ,cbar_kws={'orientation': 'vertical', 'pad' : 0.03, 'aspect': 40}, annot_kws={'size': 25} )
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=20)
#plt.ylabel('Gene/Pathway',  labelpad = 20)
plt.tick_params(axis='both', labelsize=20)
plt.xticks(rotation = 90 )'''

In [ ]:
fig_1_spear = Corr_Tab_Gly.loc[['RKIP vs BACH1','BACH1 vs KS_Mesenchymal_Gene_Signature','RKIP vs KS_Mesenchymal_Gene_Signature','BACH1 vs KS_Epithelial_Gene_Signature','RKIP vs KS_Epithelial_Gene_Signature','BACH1 vs KS_Score (Tumor)','RKIP vs KS_Score (Tumor)'],:]
fig_1_spear_star = Corr_Tab_Gly_star.loc[['RKIP vs BACH1','BACH1 vs KS_Mesenchymal_Gene_Signature','RKIP vs KS_Mesenchymal_Gene_Signature','BACH1 vs KS_Epithelial_Gene_Signature','RKIP vs KS_Epithelial_Gene_Signature','BACH1 vs KS_Score (Tumor)','RKIP vs KS_Score (Tumor)'],:]
fig_1_spear.index = ['RKIP vs BACH1','BACH1 vs KS_Mes (GS)','RKIP vs KS_Mes (GS)','BACH1 vs KS_Epi (GS)','RKIP vs KS_Epi (GS)','BACH1 vs KS_Score (Tumor)','RKIP vs KS_Score (Tumor)']

plt.figure(figsize=(25,21))
plt.rcParams['figure.dpi']= 500
#sns.set(rc={'figure.figsize':(10,10)})

#plt.title('Correlations of RKIP and BACH1 with KS Gene Signatures (Spearman coefficients)', size=20)
ax = sns.heatmap(fig_1_spear, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.9, vmin =-0.9, annot = fig_1_spear_star
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.13, 'aspect': 40}, annot_kws={'size': 25} )
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=22)
#plt.ylabel('Gene/Pathway',  labelpad = 20)
plt.tick_params(axis='both', labelsize=23)
plt.xticks(rotation = 90 )

Figure 1A

In [ ]:
fig_1_spear = Corr_Tab_Gly.loc[['RKIP vs BACH1','BACH1 vs KS_Mesenchymal_Gene_Signature','RKIP vs KS_Mesenchymal_Gene_Signature','BACH1 vs KS_Epithelial_Gene_Signature','RKIP vs KS_Epithelial_Gene_Signature','BACH1 vs KS_Score (Tumor)','RKIP vs KS_Score (Tumor)'],:]
fig_1_spear_star = Corr_Tab_Gly_star.loc[['RKIP vs BACH1','BACH1 vs KS_Mesenchymal_Gene_Signature','RKIP vs KS_Mesenchymal_Gene_Signature','BACH1 vs KS_Epithelial_Gene_Signature','RKIP vs KS_Epithelial_Gene_Signature','BACH1 vs KS_Score (Tumor)','RKIP vs KS_Score (Tumor)'],:]
fig_1_spear.index = ['RKIP vs BACH1','BACH1 vs KS_Mes (GS)','RKIP vs KS_Mes (GS)','BACH1 vs KS_Epi (GS)','RKIP vs KS_Epi (GS)','BACH1 vs KS_Score (Tumor)','RKIP vs KS_Score (Tumor)']

plt.figure(figsize=(30,20))
plt.rcParams['figure.dpi']= 1000
#sns.set(rc={'figure.figsize':(10,10)})

#plt.title('Correlations of RKIP and BACH1 with KS Gene Signatures (Spearman coefficients)', size=20)
ax = sns.heatmap(fig_1_spear, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.7, vmin =-0.7, annot = fig_1_spear_star
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.13, 'aspect': 40}, annot_kws={'size': 23} )
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=22)
#plt.ylabel('Gene/Pathway',  labelpad = 20)
plt.tick_params(axis='both', labelsize=22)
plt.xticks(rotation = 90 )
#plt.xlabel('Cancer Type', labelsize=22 )

In [ ]:
fig_1_spear = Corr_Tab_Gly.loc[['RKIP vs BACH1','BACH1 vs KS_Mesenchymal_Gene_Signature','RKIP vs KS_Mesenchymal_Gene_Signature','BACH1 vs KS_Epithelial_Gene_Signature','RKIP vs KS_Epithelial_Gene_Signature'],:]
fig_1_spear_star = Corr_Tab_Gly_star.loc[['RKIP vs BACH1','BACH1 vs KS_Mesenchymal_Gene_Signature','RKIP vs KS_Mesenchymal_Gene_Signature','BACH1 vs KS_Epithelial_Gene_Signature','RKIP vs KS_Epithelial_Gene_Signature'],:]
fig_1_spear.index = ['RKIP vs BACH1','BACH1 vs KS_Mes (GS)','RKIP vs KS_Mes (GS)','BACH1 vs KS_Epi (GS)','RKIP vs KS_Epi (GS)']

plt.figure(figsize=(30,20))
plt.rcParams['figure.dpi']= 1000
#sns.set(rc={'figure.figsize':(10,10)})

#plt.title('Correlations of RKIP and BACH1 with KS Gene Signatures (Spearman coefficients)', size=20)
ax = sns.heatmap(fig_1_spear, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.7, vmin =-0.7, annot = fig_1_spear_star
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.15, 'aspect': 40}, annot_kws={'size': 23} )
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=25)
#plt.ylabel('Gene/Pathway',  labelpad = 20)
plt.tick_params(axis='both', labelsize=25)
plt.xticks(rotation = 90 )

In [ ]:
'''fig_1b_spear = Corr_Tab_Gly.loc[[ 'RKIP vs Hallmark OXPHOS' , 'BACH1 vs Hallmark OXPHOS','RKIP vs FAO Gene Signature','BACH1 vs FAO Gene Signature','RKIP vs Hallmark Glycolysis','BACH1 vs Hallmark Glycolysis'],:]
fig_1b_spear_star = Corr_Tab_Gly_star.loc[[ 'RKIP vs Hallmark OXPHOS' , 'BACH1 vs Hallmark OXPHOS','RKIP vs FAO Gene Signature','BACH1 vs FAO Gene Signature','RKIP vs Hallmark Glycolysis','BACH1 vs Hallmark Glycolysis'],:]
fig_1b_spear.index = ['RKIP vs OXPHOS (GS)' , 'BACH1 vs OXPHOS (GS)','RKIP vs FAO (GS)','BACH1 vs FAO (GS)','RKIP vs Glycolysis (GS)','BACH1 vs Glycolysis (GS)']

plt.figure(figsize=(30,20))
plt.rcParams['figure.dpi']= 1000
#sns.set(rc={'figure.figsize':(10,10)})

#plt.title('Correlations of RKIP and BACH1 with Metabolism Gene Signatures across cancers (Spearman coefficients)', size=17)
ax = sns.heatmap(fig_1b_spear.T, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.9, vmin =-0.9, annot = fig_1b_spear_star.T
            ,cbar_kws={'orientation': 'vertical', 'pad' : 0.03, 'aspect': 40}, annot_kws={'size': 25} )


cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=20)
#plt.ylabel('Gene/Pathway',  labelpad = 20)
plt.tick_params(axis='both', labelsize=20 )
plt.xticks(rotation = 90 )'''

'''fig_1b_spear = Corr_Tab_Gly.loc[[ 'RKIP vs Hallmark OXPHOS' , 'BACH1 vs Hallmark OXPHOS','RKIP vs FAO Gene Signature','BACH1 vs FAO Gene Signature','RKIP vs Hallmark Glycolysis','BACH1 vs Hallmark Glycolysis'],:]
fig_1b_spear_star = Corr_Tab_Gly_star.loc[[ 'RKIP vs Hallmark OXPHOS' , 'BACH1 vs Hallmark OXPHOS','RKIP vs FAO Gene Signature','BACH1 vs FAO Gene Signature','RKIP vs Hallmark Glycolysis','BACH1 vs Hallmark Glycolysis'],:]
fig_1b_spear.index = ['RKIP vs OXPHOS (GS)' , 'BACH1 vs OXPHOS (GS)','RKIP vs FAO (GS)','BACH1 vs FAO (GS)','RKIP vs Glycolysis (GS)','BACH1 vs Glycolysis (GS)']

plt.figure(figsize=(30,20))
plt.rcParams['figure.dpi']= 1000
#sns.set(rc={'figure.figsize':(10,10)})

#plt.title('Correlations of RKIP and BACH1 with Metabolism Gene Signatures across cancers (Spearman coefficients)', size=17)
ax = sns.heatmap(fig_1b_spear, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.9, vmin =-0.9, annot = fig_1b_spear_star
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.14, 'aspect': 40}, annot_kws={'size': 25} )


cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=23)
#plt.ylabel('Gene/Pathway',  labelpad = 20)
plt.tick_params(axis='both', labelsize=23 )
plt.xticks(rotation = 90 )'''

Figure 1B

In [ ]:
fig_1b_spear = Corr_Tab_Gly.loc[[ 'RKIP vs Hallmark OXPHOS' , 'BACH1 vs Hallmark OXPHOS','RKIP vs FAO Gene Signature','BACH1 vs FAO Gene Signature','RKIP vs Hallmark Glycolysis','BACH1 vs Hallmark Glycolysis'],:]
fig_1b_spear_star = Corr_Tab_Gly_star.loc[[ 'RKIP vs Hallmark OXPHOS' , 'BACH1 vs Hallmark OXPHOS','RKIP vs FAO Gene Signature','BACH1 vs FAO Gene Signature','RKIP vs Hallmark Glycolysis','BACH1 vs Hallmark Glycolysis'],:]
fig_1b_spear.index = ['RKIP vs OXPHOS (GS)' , 'BACH1 vs OXPHOS (GS)','RKIP vs FAO (GS)','BACH1 vs FAO (GS)','RKIP vs Glycolysis (GS)','BACH1 vs Glycolysis (GS)']
plt.rcParams['figure.dpi']= 1000
plt.figure(figsize=(25,21))
#sns.set(rc={'figure.figsize':(10,10)})

#plt.title('Correlations of RKIP and BACH1 with Metabolism Gene Signatures across cancers (Spearman coefficients)', size=17)
ax = sns.heatmap(fig_1b_spear, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.9, vmin =-0.9, annot = fig_1b_spear_star
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.15, 'aspect': 40}, annot_kws={'size': 22} )
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=23)
#plt.ylabel('Gene/Pathway',  labelpad = 20)
plt.tick_params(axis='both', labelsize=23 )

Figure 1C

In [ ]:
fig_1b_spear = Corr_Tab_Gly.loc[[ 'RKIP vs PDL1 Signature' , 'BACH1 vs PDL1 Signature'],:]
fig_1b_spear_star = Corr_Tab_Gly_star.loc[['RKIP vs PDL1 Signature' , 'BACH1 vs PDL1 Signature'],:]
fig_1b_spear.index = ['RKIP vs PDL1 (GS)' , 'BACH1 vs PDL1 (GS)']
plt.rcParams['figure.dpi']= 1000
plt.figure(figsize=(26,20))
#sns.set(rc={'figure.figsize':(10,10)})

#plt.title('Pan Cancer Analysis of RKIP and BACH1 with PDL1 Signature (Spearman coefficients)', size=17)
ax = sns.heatmap(fig_1b_spear, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.9, vmin =-0.9, annot = fig_1b_spear_star
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.15, 'aspect': 40}, annot_kws={'size': 22} )
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=22)
#plt.ylabel('Gene/Pathway',  labelpad = 20)
plt.tick_params(axis='both', labelsize=23)
plt.xticks( rotation=90)

Figure 1D

In [ ]:
fig_1d_spear = Corr_Tab_Gly.loc[[ 'RKIP vs GenSig_Ferroptosis_Immune_45' , 'BACH1 vs GenSig_Ferroptosis_Immune_45'],:]
fig_1d_spear_star = Corr_Tab_Gly_star.loc[['RKIP vs GenSig_Ferroptosis_Immune_45' , 'BACH1 vs GenSig_Ferroptosis_Immune_45'],:]
fig_1d_spear.index = ['RKIP vs Ferroptosis (GS)' , 'BACH1 vs Ferroptosis (GS)']
plt.rcParams['figure.dpi']= 1000
plt.figure(figsize=(26,20))
#sns.set(rc={'figure.figsize':(10,10)})

#plt.title('Correlation of RKIP and BACH1 with Metabolic Signatures (Spearman coefficients)', size=17)
ax = sns.heatmap(fig_1d_spear, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.9, vmin =-0.9, annot = fig_1d_spear_star
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.17, 'aspect': 40}, annot_kws={'size':20} )
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=22)
#plt.ylabel('Gene/Pathway',  labelpad = 20)
plt.tick_params(axis='both', labelsize=22)

Supplementary Analysis

In [ ]:
fig_s1a_spear = Corr_Tab_Gly.loc[[ 'BACH1 vs 76GS_Score' , 'RKIP vs 76GS_Score','76GS_Score vs KS_Score (Tumor)'],:]
fig_s1a_spear_star = Corr_Tab_Gly_star.loc[[ 'BACH1 vs 76GS_Score' , 'RKIP vs 76GS_Score','76GS_Score vs KS_Score (Tumor)'],:]
fig_s1a_spear.index = [ 'BACH1 vs 76GS_Score' , 'RKIP vs 76GS_Score','76GS_Score vs KS_Score (Tumor)']
plt.rcParams['figure.dpi']= 400
plt.figure(figsize=(25,20))
#sns.set(rc={'figure.figsize':(10,10)})

#plt.title('Correlation of RKIP and BACH1 with EMT scores (Spearman coefficients)', size=17)
ax = sns.heatmap(fig_s1a_spear, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.9, vmin =-0.9, annot = fig_s1a_spear_star
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.15, 'aspect': 40}, annot_kws={'size': 20} )
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=20)
#plt.ylabel('Gene/Pathway',  labelpad = 20)
plt.tick_params(axis='both', labelsize=20)

Supplementary 1A

In [ ]:
fig_s1c_spear = Corr_Tab_Gly.loc[[ 'Hallmark_OXPHOS vs Hallmark_Glycolysis' , 'FAO_Gene_Signature vs Hallmark_OXPHOS','FAO_Gene_Signature vs Hallmark_Glycolysis'],:]
fig_s1c_spear_star = Corr_Tab_Gly_star.loc[[ 'Hallmark_OXPHOS vs Hallmark_Glycolysis' , 'FAO_Gene_Signature vs Hallmark_OXPHOS','FAO_Gene_Signature vs Hallmark_Glycolysis'],:]
fig_s1c_spear.index = [  'OXPHOS (GS) vs Glycolysis (GS)' , 'FAO (GS) vs OXPHOS (GS)','FAO (GS) vs Glycolysis (GS)']
plt.rcParams['figure.dpi']= 1000
plt.figure(figsize=(20,15))
#sns.set(rc={'figure.figsize':(10,10)})

#plt.title('Correlation of RKIP and BACH1 with Metabolic Signatures (Spearman coefficients)', size=17)
ax = sns.heatmap(fig_s1c_spear, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.9, vmin =-0.9, annot = fig_s1c_spear_star
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.17, 'aspect': 40}, annot_kws={'size':20} )
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=22)
#plt.ylabel('Gene/Pathway',  labelpad = 20)
plt.tick_params(axis='both', labelsize=22)

Supplemantary S2B

In [ ]:
fig_s2c_spear = Corr_Tab_Gly.loc[[ 'RKIP vs BACH1' , 'BACH1_Metastasis_GenSig vs RKIP','BACH1_Metastasis_GenSig vs BACH1','RKIP_Metastasis_GenSig vs RKIP','RKIP_Metastasis_GenSig vs BACH1','RKIP_Metastasis_GenSig vs BACH1_Metastasis_GenSig'],:]
fig_s2c_spear_star = Corr_Tab_Gly_star.loc[[ 'RKIP vs BACH1' , 'BACH1_Metastasis_GenSig vs RKIP','BACH1_Metastasis_GenSig vs BACH1','RKIP_Metastasis_GenSig vs RKIP','RKIP_Metastasis_GenSig vs BACH1','RKIP_Metastasis_GenSig vs BACH1_Metastasis_GenSig'],:]
fig_s2c_spear.index = [ 'RKIP vs BACH1' , 'BPMS (GS) vs RKIP','BPMS (GS) vs BACH1','RPMS (GS) vs RKIP','RPMS (GS) vs BACH1','RPMS (GS) vs BPMS (GS) ']
plt.rcParams['figure.dpi']= 1000
plt.figure(figsize=(20,15))
#sns.set(rc={'figure.figsize':(10,10)})

#plt.title('Correlation of RKIP and BACH1 with Metastasis Signatures scores (Spearman coefficients)', size=17)
ax = sns.heatmap(fig_s2c_spear, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.9, vmin =-0.9, annot = fig_s2c_spear_star
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.17, 'aspect': 40}, annot_kws={'size': 15} )
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=20)
#plt.ylabel('Gene/Pathway',  labelpad = 20)
plt.tick_params(axis='both', labelsize=20)

In [ ]:
'''corr_scatter(TCGA_PCPG_scored.T,'PEBP1','CD274', title = 'TCGA (PCPG) ( 17 datasets ) ')
corr_scatter(TCGA_PCPG_scored.T,'BACH1','CD274', title = 'TCGA (PCPG) ( 17 datasets ) ')'''

In [ ]:
fig_s1c_spear = Corr_Tab_Gly.loc[[ 'RKIP vs CD274(PDL1)' , 'BACH1 vs CD274(PDL1)'],:]
fig_s1c_spear_star = Corr_Tab_Gly_star.loc[[ 'RKIP vs CD274(PDL1)'  ,'BACH1 vs CD274(PDL1)'],:]
fig_s1c_spear.index = [  'RKIP vs PDL1' , 'BACH1 vs PDL1']
plt.rcParams['figure.dpi']= 1000
plt.figure(figsize=(25,20))
#sns.set(rc={'figure.figsize':(10,10)})

#plt.title('Correlation of RKIP and BACH1 with PDL1 (Spearman coefficients)', size=17)
ax = sns.heatmap(fig_s1c_spear, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.9, vmin =-0.9, annot = fig_s1c_spear_star
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.14, 'aspect': 40}, annot_kws={'size': 22} )
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=22)
#plt.ylabel('Gene/Pathway',  labelpad = 20)
plt.tick_params(axis='both', labelsize=22)

Ferroptosis

In [ ]:
'''fig_ferr_spear = Corr_Tab_Gly.loc[[ 'RKIP vs NRF2' , 'BACH1 vs NRF2','RKIP vs NRF2_Gene_Signature','BACH1 vs NRF2_Gene_Signature','RKIP vs GenSig_Ferroptosis_6'
                                  ,'BACH1 vs GenSig_Ferroptosis_6', 'RKIP vs GenSig_Ferroptosis_10_LBC','BACH1 vs GenSig_Ferroptosis_10_LBC',
                                  'RKIP vs GenSig_Ferroptosis_Immune_45', 'BACH1 vs GenSig_Ferroptosis_Immune_45'],:]
fig_ferr_spear_star = Corr_Tab_Gly_star.loc[[ 'RKIP vs NRF2' , 'BACH1 vs NRF2','RKIP vs NRF2_Gene_Signature','BACH1 vs NRF2_Gene_Signature','RKIP vs GenSig_Ferroptosis_6'
                                  ,'BACH1 vs GenSig_Ferroptosis_6', 'RKIP vs GenSig_Ferroptosis_10_LBC','BACH1 vs GenSig_Ferroptosis_10_LBC',
                                  'RKIP vs GenSig_Ferroptosis_Immune_45', 'BACH1 vs GenSig_Ferroptosis_Immune_45'],:]
#fig_s1c_spear.index = [  'RKIP vs PDL1' , 'BACH1 vs PDL1']

plt.figure(figsize=(20,15))
#sns.set(rc={'figure.figsize':(10,10)})

plt.title('Correlation of RKIP and BACH1 with Ferroptosis and NRF2 (Spearman coefficients)', size=17)
ax = sns.heatmap(fig_ferr_spear, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.9, vmin =-0.9, annot = fig_ferr_spear_star
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.13, 'aspect': 40}, annot_kws={'size': 15} )
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=16)
#plt.ylabel('Gene/Pathway',  labelpad = 20)
plt.tick_params(axis='both', labelsize=15)'''

In [ ]:
  
 '''BACH1_Metastasis.gmt   Corr_Tab_Gly.loc['RKIP vs NRF2', y]  = stats.spearmanr( data['PEBP1'], data['NFE2L2'])[i]
    Corr_Tab_Gly.loc['BACH1 vs NRF2', y]  = stats.spearmanr( data['BACH1'], data['NFE2L2'])[i]
    Corr_Tab_Gly.loc['RKIP vs NRF2_Gene_Signature', y]  = stats.spearmanr( data['PEBP1'], data['NRF2_Gene_Signature'])[i]
    Corr_Tab_Gly.loc['BACH1 vs NRF2_Gene_Signature', y]  = stats.spearmanr( data['BACH1'], data['NRF2_Gene_Signature'])[i]
    Corr_Tab_Gly.loc['RKIP vs GenSig_Ferroptosis_6', y]  = stats.spearmanr( data['PEBP1'], data['GenSig_Ferroptosis_6'])[i]
    Corr_Tab_Gly.loc['BACH1 vs GenSig_Ferroptosis_6', y]  = stats.spearmanr( data['BACH1'], data['GenSig_Ferroptosis_6'])[i]
    Corr_Tab_Gly.loc['RKIP vs GenSig_Ferroptosis_10_LBC', y]  = stats.spearmanr( data['PEBP1'], data['GenSig_Ferroptosis_10_LBC'])[i]
    Corr_Tab_Gly.loc['BACH1 vs GenSig_Ferroptosis_10_LBC', y]  = stats.spearmanr( data['BACH1'], data['GenSig_Ferroptosis_10_LBC'])[i]
    Corr_Tab_Gly.loc['RKIP vs GenSig_Ferroptosis_Immune_45', y]  = stats.spearmanr( data['PEBP1'], data['GenSig_Ferroptosis_Immune_45'])[i]
    Corr_Tab_Gly.loc['BACH1 vs GenSig_Ferroptosis_Immune_45', y]  = stats.spearmanr( data['BACH1'], data['GenSig_Ferroptosis_Immune_45'])[i]
'''


In [ ]:
Ba=pd.read_csv('BACH1_Survival.csv',index_col=0)
RKI=pd.read_csv('RKIP_Survival.csv', index_col=0)

In [ ]:
da=pd.DataFrame([[0]*len(Ba.iloc[1,:]),[0]*len(Ba.iloc[1,:])])
da.index=['BACH1','RKIP']
da.columns=Ba.columns
pa=da.copy()
da

In [ ]:
da.loc['BACH1',:]=Ba.iloc[0,:]
da.loc['RKIP',:]=RKI.iloc[0,:]

In [ ]:
pa.loc['BACH1',:]=Ba.iloc[0,:]
pa.loc['RKIP',:]=RKI.iloc[0,:]

In [ ]:
pa_star=pa.copy()

for i in range(len(pa_star)):
    for j in range(len(pa_star.iloc[0,:])):
        pa.iloc[i,j]=float(pa.iloc[i,j])

        if pa.iloc[i,j] <= 0.05:
            pa_star.iloc[i,j]= "*"
        else:
            pa_star.iloc[ i ,j ] = ""
pa_star


In [ ]:
for i in range(len(da)):
    for j in range(len(da.iloc[0,:])):
        da.iloc[i,j]=float(da.iloc[i,j])
        

In [ ]:
TCGA_ACC_scored = pd.read_csv('Data_Figure1_2/TCGA_ACC.csv', index_col=0)


In [ ]:
len(da.columns)
a = ['ACC', 'BLCA', 'BRCA', 'CESC', 'CHOL',
              'COAD','DLBC', 'ESCA', 'GBM', 
              'HNSC', 'KICH', 'KIRC', 'KIRP', 'LAML','LGG', 'LIHC', 'LUAD',  'LUSC',
              'MESO', 'OV', 'PAAD', 'PCPG', 'PRAD','READ', 'SARC', 'SKCM', 'STAD', 'TGCT',
              'THCA', 'THYM', 'UCEC', 'UCS', 'UVM']
da.columns

In [ ]:

da.columns=a
plt.rcParams['figure.dpi']= 1000
plt.figure(figsize=(25,20))

ax = sns.heatmap(da, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 2, vmin =-2                 
                 , annot=pa_star, cbar_kws={'orientation': 'horizontal', 'pad' : 0.11, 'aspect': 40}, annot_kws={'size': 22} )
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=22)
cbar.set_label('log2HR', size=22,fontweight='bold')
#plt.ylabel('Gene/Pathway',  labelpad = 20)
plt.tick_params(axis='both', labelsize=23)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
#plt.xlabel('Cancer Type',size=23,fontweight='bold')


In [ ]:
'''fig_1b_spear = Corr_Tab_Gly.loc[[ 'RKIP vs PDL1 Signature' , 'BACH1 vs PDL1 Signature'],:]
fig_1b_spear_star = Corr_Tab_Gly_star.loc[['RKIP vs PDL1 Signature' , 'BACH1 vs PDL1 Signature'],:]
fig_1b_spear.index = ['RKIP vs PDL1 (GS)' , 'BACH1 vs PDL1 (GS)']
plt.rcParams['figure.dpi']= 1000
plt.figure(figsize=(26,20))
#sns.set(rc={'figure.figsize':(10,10)})

#plt.title('Pan Cancer Analysis of RKIP and BACH1 with PDL1 Signature (Spearman coefficients)', size=17)
ax = sns.heatmap(fig_1b_spear, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.7, vmin =-0.7, annot = fig_1b_spear_star
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.15, 'aspect': 40}, annot_kws={'size': 22} )
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=22)
#plt.ylabel('Gene/Pathway',  labelpad = 20)
plt.tick_params(axis='both', labelsize=23)
plt.xticks( rotation=90)

fig_ferr_spear = Corr_Tab_Gly.loc[[ 'RKIP vs NRF2' , 'BACH1 vs NRF2','RKIP vs NRF2_Gene_Signature','BACH1 vs NRF2_Gene_Signature','RKIP vs GenSig_Ferroptosis_6'
                                  ,'BACH1 vs GenSig_Ferroptosis_6', 'RKIP vs GenSig_Ferroptosis_10_LBC','BACH1 vs GenSig_Ferroptosis_10_LBC',
                                  'RKIP vs GenSig_Ferroptosis_Immune_45', 'BACH1 vs GenSig_Ferroptosis_Immune_45'],:]
fig_ferr_spear_star = Corr_Tab_Gly_star.loc[[ 'RKIP vs NRF2' , 'BACH1 vs NRF2','RKIP vs NRF2_Gene_Signature','BACH1 vs NRF2_Gene_Signature','RKIP vs GenSig_Ferroptosis_6'
                                  ,'BACH1 vs GenSig_Ferroptosis_6', 'RKIP vs GenSig_Ferroptosis_10_LBC','BACH1 vs GenSig_Ferroptosis_10_LBC',
                                  'RKIP vs GenSig_Ferroptosis_Immune_45', 'BACH1 vs GenSig_Ferroptosis_Immune_45'],:]
#fig_s1c_spear.index = [  'RKIP vs PDL1' , 'BACH1 vs PDL1']

plt.figure(figsize=(20,15))
#sns.set(rc={'figure.figsize':(10,10)})

plt.title('Correlation of RKIP and BACH1 with Ferroptosis and NRF2 (Spearman coefficients)', size=17)
ax = sns.heatmap(fig_ferr_spear, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.9, vmin =-0.9, annot = fig_ferr_spear_star
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.13, 'aspect': 40}, annot_kws={'size': 15} )
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=16)
#plt.ylabel('Gene/Pathway',  labelpad = 20)
plt.tick_params(axis='both', labelsize=15)'''

In [ ]:
'''
len(da.iloc[0,:])
ax = sns.heatmap(fig_ferr_spear, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.9, vmin =-0.9, annot = fig_ferr_spear_star
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.13, 'aspect': 40}, annot_kws={'size': 15} )
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=16)'''